# Secao4.3 FDC Sazonalidade

> Notebook organizado para reprodutibilidade. Edite apenas a célula **CONFIGURAÇÕES**.

In [ ]:
from pathlib import Path
import os

# CONFIGURAÇÕES (edite se necessário)
# A pasta raiz do projeto (por padrão, a pasta acima de /notebooks)
ROOT = Path(os.getenv('CLIMBRA_PROJECT_ROOT', Path.cwd().parent)).resolve()
DATA_DIR = ROOT / 'data'
RAW_DIR  = DATA_DIR / '00_raw'
INT_DIR  = DATA_DIR / '01_intermediate'
FINAL_DIR= DATA_DIR / '02_final'
OUT_DIR  = ROOT / 'outputs'
FIG_DIR  = OUT_DIR / 'figures'
TAB_DIR  = OUT_DIR / 'tables'

for d in [RAW_DIR, INT_DIR, FINAL_DIR, FIG_DIR, TAB_DIR]:
    d.mkdir(parents=True, exist_ok=True)


In [ ]:
# -*- coding: utf-8 -*-
"""
===============================================================================
EXPORTAÇÃO: 1 GeoPackage (.gpkg) por camada (1 layer por arquivo)

- Corrige CRS ausente (heurística por bounds ou CRS fixo por camada)
- Reprojeta tudo para EPSG:31982 (SIRGAS 2000 / UTM 22S)
- Saneia colunas problemáticas (ex.: 'fid') para evitar FieldError
- Salva um .gpkg por camada

PLUS (hidrografia):
- Filtra ordem <= ORDEM_MAX
- Recorte rápido ao limite das sub-bacias (union) usando MASK vetorizado (sjoin)
  (SEM clip geométrico — mais rápido e suficiente para plotagem)

===============================================================================
"""

from __future__ import annotations

from pathlib import Path
from typing import Tuple, Dict

import pandas as pd
import geopandas as gpd


# =============================================================================
# ENTRADAS
# =============================================================================
INPUTS = {
    "minis_mgb": Path(r"E:\IGUAÇU_OTTO\6_Calibração\minis_mgb.shp"),
    "subbacias": Path(r"E:\IGUAÇU_OTTO\Shp\Subbacias.shp"),
    "cidades": Path(
        r"G:\Meu Drive\2_MESTRADO\1_Dissertação\Figuras\20250516_SHAPES_FIGURA\GEOFT_CIDADE_2016.shp"
    ),
    "estacoes": Path(r"E:\IGUAÇU_OTTO\3_Estações FLU\Estações_Alto_IG.shp"),
    "hidrografia": Path(r"E:\IGUAÇU_OTTO\Shp\Hidrografia.shp"),
}

# =============================================================================
# SAÍDA
# =============================================================================
OUT_DIR = Path(r"E:\Base_Cartografica_GPKG")
CRS_FINAL_EPSG = 31982

# Hidrografia
ORDEM_MAX = 5
CAMPO_ORDEM_PREFERIDO = "nuordemcda"  # se não existir, o código tenta detectar automaticamente

# Se quiser FORÇAR CRS de origem quando estiver ausente (mais seguro):
CRS_ORIGEM_FIXO_SEM_CRS: Dict[str, int] = {
    # "estacoes": 31982,
    # "hidrografia": 31982,
}


# =============================================================================
# FUNÇÕES
# =============================================================================

def ensure_dir(p: Path) -> None:
    p.mkdir(parents=True, exist_ok=True)


def bounds_look_geographic(bounds: Tuple[float, float, float, float]) -> bool:
    minx, miny, maxx, maxy = bounds
    return (
        -180 <= minx <= 180 and -180 <= maxx <= 180 and
        -90 <= miny <= 90 and -90 <= maxy <= 90
    )


def guess_crs_if_missing(gdf: gpd.GeoDataFrame) -> int:
    b = tuple(gdf.total_bounds)
    return 4326 if bounds_look_geographic(b) else 31982


def sanitize_gdf(gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """
    Saneia campos para escrita em GeoPackage via pyogrio:
    - renomeia colunas que costumam conflitar (fid, ogc_fid)
    - remove colunas duplicadas
    - remove geometrias nulas
    """
    gdf = gdf.copy()
    gdf = gdf[~gdf.geometry.isna()].copy()

    def _rename_if_exists(col_lower: str, new_base: str):
        nonlocal gdf
        for c in list(gdf.columns):
            if c.lower() == col_lower:
                new_name = new_base
                i = 1
                while new_name in gdf.columns:
                    i += 1
                    new_name = f"{new_base}{i}"
                gdf = gdf.rename(columns={c: new_name})

    _rename_if_exists("fid", "fid_src")
    _rename_if_exists("ogc_fid", "ogc_fid_src")

    if not gdf.columns.is_unique:
        gdf = gdf.loc[:, ~gdf.columns.duplicated()].copy()

    return gdf


def load_fix_reproject(path: Path, layer_name: str) -> gpd.GeoDataFrame:
    gdf = gpd.read_file(path)

    if gdf.crs is None:
        if layer_name in CRS_ORIGEM_FIXO_SEM_CRS:
            src = CRS_ORIGEM_FIXO_SEM_CRS[layer_name]
            print(f"[CRS] {path.name}: CRS ausente -> FORÇADO EPSG:{src}")
        else:
            src = guess_crs_if_missing(gdf)
            print(f"[CRS] {path.name}: CRS ausente -> heurística EPSG:{src}")
        gdf = gdf.set_crs(epsg=src, allow_override=True)

    if gdf.crs.to_epsg() != CRS_FINAL_EPSG:
        gdf = gdf.to_crs(epsg=CRS_FINAL_EPSG)
        print(f"[CRS] {path.name}: reprojetado -> EPSG:{CRS_FINAL_EPSG}")
    else:
        print(f"[CRS] {path.name}: já está -> EPSG:{CRS_FINAL_EPSG}")

    return sanitize_gdf(gdf)


def detectar_campo_ordem(gdf_h: gpd.GeoDataFrame, preferido: str | None) -> str:
    cols = list(gdf_h.columns)
    if preferido and preferido in cols:
        return preferido

    lower_map = {c.lower(): c for c in cols}
    candidatos = [
        "nuordemdca", "nuordemcda", "nuordem", "ordem", "order", "ord",
        "strahler", "str_order", "ord_strah"
    ]
    for c in candidatos:
        if c.lower() in lower_map:
            return lower_map[c.lower()]

    heur = [c for c in cols if ("ord" in c.lower()) or ("strah" in c.lower())]
    if heur:
        return heur[0]

    raise KeyError("Não encontrei campo de ordem na hidrografia. Inspecione os campos do arquivo.")


def filtrar_recortar_hidrografia_rapido(
    gdf_h: gpd.GeoDataFrame,
    gdf_sub: gpd.GeoDataFrame,
    ordem_max: int,
    campo_ordem_preferido: str | None,
) -> gpd.GeoDataFrame:
    """
    Hidrografia para plot (rápido):
    1) Filtra ordem <= ordem_max
    2) Recorte rápido por:
       - bbox do polígono união
       - spatial join (predicate intersects) com a máscara (sem recortar geometria)
    """
    campo = detectar_campo_ordem(gdf_h, campo_ordem_preferido)

    gdf = gdf_h.copy()
    gdf["_ord"] = pd.to_numeric(gdf[campo], errors="coerce").fillna(-1).astype(int)
    gdf = gdf[(gdf["_ord"] >= 1) & (gdf["_ord"] <= ordem_max)].copy()

    clip_geom = gdf_sub.unary_union.buffer(0)  # robustez
    minx, miny, maxx, maxy = clip_geom.bounds

    # bbox primeiro (reduz muito)
    gdf = gdf.cx[minx:maxx, miny:maxy].copy()

    # máscara como GeoDataFrame
    mask = gpd.GeoDataFrame(geometry=[clip_geom], crs=gdf.crs)

    # spatial join vetorizado (rápido)
    gdf = gpd.sjoin(gdf, mask, how="inner", predicate="intersects")
    gdf = gdf.drop(columns=["index_right", "_ord"], errors="ignore")

    return gdf


def write_single_layer_gpkg(gdf: gpd.GeoDataFrame, out_gpkg: Path, layer: str) -> None:
    if out_gpkg.exists():
        out_gpkg.unlink()

    gdf2 = sanitize_gdf(gdf)

    # NÃO especificar engine -> usa pyogrio automaticamente no seu ambiente
    gdf2.to_file(out_gpkg, layer=layer, driver="GPKG", index=False)
    print(f"[OK] {out_gpkg.name} | layer='{layer}' | feats={len(gdf2)}")


# =============================================================================
# MAIN
# =============================================================================

def main():
    ensure_dir(OUT_DIR)

    # 1) Carrega subbacias primeiro (para máscara do recorte)
    if not INPUTS["subbacias"].exists():
        raise FileNotFoundError(f"Arquivo não encontrado: {INPUTS['subbacias']}")
    gdf_sub = load_fix_reproject(INPUTS["subbacias"], "subbacias")

    # 2) Exporta camadas (hidrografia já sai filtrada/recortada rápida)
    for layer_name, shp_path in INPUTS.items():
        if not shp_path.exists():
            raise FileNotFoundError(f"Arquivo não encontrado: {shp_path}")

        gdf = load_fix_reproject(shp_path, layer_name)

        if layer_name == "hidrografia":
            gdf = filtrar_recortar_hidrografia_rapido(
                gdf_h=gdf,
                gdf_sub=gdf_sub,
                ordem_max=ORDEM_MAX,
                campo_ordem_preferido=CAMPO_ORDEM_PREFERIDO,
            )
            print(f"[HIDRO] Hidrografia pronta para plot: ordem<= {ORDEM_MAX} + recorte rápido (sjoin)")

        out_gpkg = OUT_DIR / f"{layer_name}.gpkg"
        write_single_layer_gpkg(gdf, out_gpkg, layer=layer_name)

    print(f"\n✅ Concluído! GPKGs salvos em: {OUT_DIR}")


if __name__ == "__main__":
    main()

In [ ]:
# ------------------------------------------------------------
# Script: FDC por Estação — Observado (REF) + Simulado Futuro (Modelos + Ensemble)
#          (Ensemble = média por posição/quantil; NÃO média diária no tempo)
#
# O QUE FOI CORRIGIDO:
# - O "ENSEMBLE_MEAN" agora é calculado no espaço da excedência:
#   1) Para cada horizonte, calcula-se a FDC (quantis) de cada modelo
#   2) Para cada excedência (posição), calcula-se a média entre modelos
#
# Vantagem: evita o artefato de elevar as mínimas por dessincronização temporal
# entre modelos (média diária antes de ordenar).
# ------------------------------------------------------------

from __future__ import annotations

from pathlib import Path
from typing import Dict, List, Tuple, Optional

import numpy as np
import pandas as pd

# =============================================================================
# CONFIGURAÇÕES
# =============================================================================

# Projeções
RAIZ_PROJECOES = Path(r"E:\IGUAÇU_OTTO\7_Projeções_ssp585 - Clima")
SHIFT_ANOS = 80

# Observado (ascii_mgb)
OBS_DIR = Path(r"E:\IGUAÇU_OTTO\3_Estações FLU\Input\ascii_mgb")

# Mapeamento estação -> minibacia
MAPEAMENTO_CSV = Path(r"E:\IGUAÇU_OTTO\3_Estações FLU\Estações_mini.csv")
SEP_MAP = ";"

# Saída
OUT_DIR = Path(r"E:\RESULTADOS_AB2\SSP5-85\FDC_Projecoes_ssp585")
SEP_OUT = ";"
ENC_OUT = "utf-8"

# Excedência
EXC_PCT = np.arange(0.0, 100.0 + 1e-9, 1.0)

# REF observada
REF_OBS = ("1980-01-01", "2023-12-31")

# 4 horizontes futuros
HORIZONTES: Dict[str, Tuple[str, str]] = {
    "CURTO_2015_2040": ("2015-01-01", "2040-12-31"),
    "MEDIO_2041_2070": ("2041-01-01", "2070-12-31"),
    "LONGO_2071_2100": ("2071-01-01", "2100-12-31"),
    "TOTAL_2015_2100": ("2015-01-01", "2100-12-31"),
}

# =============================================================================
# FUNÇÕES
# =============================================================================

def garantir_pasta(p: Path) -> None:
    p.mkdir(parents=True, exist_ok=True)


def listar_modelos(raiz: Path) -> List[Path]:
    modelos: List[Path] = []
    for p in sorted(raiz.iterdir()):
        if p.is_dir() and (p / "Output").exists():
            modelos.append(p)
    return modelos


def ler_txt_mgb(caminho: Path) -> pd.Series:
    """
    Lê arquivo MGB (dia mes ano valor). Trata -1 como NaN.
    """
    df = pd.read_csv(
        caminho,
        sep=r"\s+",
        header=None,
        names=["dia", "mes", "ano", "valor"],
        dtype=str,
        engine="python",
    )
    df["valor"] = df["valor"].str.replace(",", "", regex=False).astype(float)
    df.loc[df["valor"] == -1, "valor"] = np.nan

    df["data"] = pd.to_datetime(
        df.rename(columns={"ano": "year", "mes": "month", "dia": "day"})[["year", "month", "day"]],
        errors="coerce",
    )
    df = df.dropna(subset=["data"]).set_index("data").sort_index()
    s = df["valor"]
    s.name = caminho.stem
    return s


def aplicar_shift_anos(s: pd.Series, shift_anos: int) -> pd.Series:
    s2 = s.copy()
    s2.index = s2.index + pd.DateOffset(years=shift_anos)
    return s2


def recortar(s: pd.Series, ini: str, fim: str) -> pd.Series:
    return s.loc[(s.index >= pd.Timestamp(ini)) & (s.index <= pd.Timestamp(fim))]


def calcular_fdc(s: pd.Series, exc_pct: np.ndarray) -> pd.DataFrame:
    """
    FDC via quantis:
      excedência p% -> quantil (1 - p)
    """
    x = s.dropna().values
    if x.size == 0:
        return pd.DataFrame({"exc": exc_pct, "q": np.nan})

    p = exc_pct / 100.0
    q = np.quantile(x, 1.0 - p)
    return pd.DataFrame({"exc": exc_pct, "q": q})


def ensemble_fdc_por_posicao(
    fdc_por_modelo: Dict[str, pd.DataFrame],
    metodo: str = "mean",
) -> Optional[pd.DataFrame]:
    """
    Recebe dict: modelo -> DataFrame FDC com colunas ['exc','q'] (mesma grade de exc)
    Retorna DataFrame com ['exc','q'] do ensemble por posição (por excedência).

    metodo:
      - "mean": média entre modelos em cada excedência
      - "median": mediana entre modelos em cada excedência (mais robusta)
    """
    if not fdc_por_modelo:
        return None

    # monta matriz (linhas=excedências, colunas=modelos)
    cols = {}
    exc_ref = None
    for m, df in fdc_por_modelo.items():
        if exc_ref is None:
            exc_ref = df["exc"].to_numpy()
        cols[m] = df["q"].to_numpy()

    mat = pd.DataFrame(cols)
    if metodo.lower() == "median":
        q_ens = mat.median(axis=1, skipna=True).to_numpy()
    else:
        q_ens = mat.mean(axis=1, skipna=True).to_numpy()

    out = pd.DataFrame({"exc": exc_ref, "q": q_ens})
    return out


# =============================================================================
# MAIN
# =============================================================================

def main() -> None:
    garantir_pasta(OUT_DIR)

    map_df = pd.read_csv(MAPEAMENTO_CSV, sep=SEP_MAP, dtype=str).fillna("")
    obrig = {"estacao_obs", "codigo_mini", "nome_estacao"}
    if not obrig.issubset(map_df.columns):
        raise ValueError(f"CSV de mapeamento deve conter colunas: {sorted(obrig)}")

    modelos = listar_modelos(RAIZ_PROJECOES)
    if not modelos:
        raise FileNotFoundError(f"Não encontrei pastas de modelos com Output/ em {RAIZ_PROJECOES}")

    all_rows: List[pd.DataFrame] = []

    for _, row in map_df.iterrows():
        estacao = row["estacao_obs"].strip()
        mini = row["codigo_mini"].strip()
        nome = row["nome_estacao"].strip()

        if not estacao or not mini:
            continue

        # -----------------------------
        # Observado (REF)
        # -----------------------------
        arq_obs = OBS_DIR / f"{estacao}.txt"
        if not arq_obs.exists():
            print(f"⚠️ Observado não encontrado: {arq_obs.name}. Pulando {estacao}.")
            continue

        s_obs = ler_txt_mgb(arq_obs)
        s_obs_ref = recortar(s_obs, *REF_OBS)
        fdc_obs_ref = calcular_fdc(s_obs_ref, EXC_PCT).rename(columns={"q": "q_obs_ref"})
        fdc_obs_ref.insert(0, "horizonte", "REF_1980_2023")
        fdc_obs_ref.insert(0, "modelo", "OBS")

        # -----------------------------
        # Simulado por modelo (futuro)
        # -----------------------------
        sims: Dict[str, pd.Series] = {}
        for pasta_modelo in modelos:
            mname = pasta_modelo.name
            arq_sim = pasta_modelo / "Output" / f"SIM_MC_{mini}.TXT"
            if not arq_sim.exists():
                continue

            s = ler_txt_mgb(arq_sim)
            s = aplicar_shift_anos(s, SHIFT_ANOS)
            sims[mname] = s

        if not sims:
            print(f"⚠️ Nenhum simulado encontrado para {estacao} (SIM_MC_{mini}).")
            continue

        saidas: List[pd.DataFrame] = []

        # -----------------------------
        # FDCs dos modelos por horizonte
        # -----------------------------
        for horiz, (ini, fim) in HORIZONTES.items():
            # 1) calcula FDC de cada modelo nesse horizonte
            fdc_por_modelo: Dict[str, pd.DataFrame] = {}
            for mname, s in sims.items():
                s_h = recortar(s, ini, fim)
                fdc_m = calcular_fdc(s_h, EXC_PCT)
                fdc_por_modelo[mname] = fdc_m

                # salva a curva individual
                fdc_m_out = fdc_m.rename(columns={"q": "q_sim"}).copy()
                fdc_m_out.insert(0, "horizonte", horiz)
                fdc_m_out.insert(0, "modelo", mname)
                saidas.append(fdc_m_out)

            # 2) ensemble por posição (média dos quantis)
            fdc_ens = ensemble_fdc_por_posicao(fdc_por_modelo, metodo="mean")
            if fdc_ens is not None:
                fdc_ens_out = fdc_ens.rename(columns={"q": "q_sim"}).copy()
                fdc_ens_out.insert(0, "horizonte", horiz)
                fdc_ens_out.insert(0, "modelo", "ENSEMBLE_MEAN_POS")
                saidas.append(fdc_ens_out)

        df_sim = pd.concat(saidas, ignore_index=True)

        # -----------------------------
        # Junta curva observada (REF) por excedência
        # -----------------------------
        df_sim = df_sim.merge(
            fdc_obs_ref[["exc", "q_obs_ref"]],
            on="exc",
            how="left",
        )

        # metadados
        df_sim.insert(0, "codigo_mini", mini)
        df_sim.insert(0, "nome_estacao", nome)
        df_sim.insert(0, "estacao_obs", estacao)

        # Também salva a FDC observada REF como bloco próprio
        df_obs_out = fdc_obs_ref.copy()
        df_obs_out.insert(0, "codigo_mini", mini)
        df_obs_out.insert(0, "nome_estacao", nome)
        df_obs_out.insert(0, "estacao_obs", estacao)

        # concatena obs_ref (como bloco separado) + futuros (ensure colunas compatíveis)
        df_obs_block = df_obs_out.assign(q_sim=np.nan)
        df_out = pd.concat([df_obs_block, df_sim], ignore_index=True)

        out_file = OUT_DIR / f"FDC_{estacao}_obs_ref_e_futuro.csv"
        df_out.to_csv(out_file, sep=SEP_OUT, index=False, encoding=ENC_OUT)

        all_rows.append(df_out)
        print(f"✅ OK: {estacao} | modelos={len(sims)} | {out_file.name}")

    if all_rows:
        df_all = pd.concat(all_rows, ignore_index=True)
        out_all = OUT_DIR / "FDC_consolidado_obs_ref_e_futuro.csv"
        df_all.to_csv(out_all, sep=SEP_OUT, index=False, encoding=ENC_OUT)
        print(f"\n✅ Consolidado salvo: {out_all}")


if __name__ == "__main__":
    main()

In [ ]:
# -*- coding: utf-8 -*-
"""
===============================================================================
SAZONALIDADE — modelos individuais + ensemble + observado

Saídas:
1) CSV consolidado (formato longo | climatologia mensal por horizonte):
   SAZONALIDADE_mensal_consolidada.csv

2) CSV individual por estação (formato longo | climatologia mensal por horizonte):
   por_estacao/SAZONALIDADE_<estacao_obs>.csv

3) CSV PADRÃO (formato wide, para plot rápido no mapa):
   SAZONALIDADE_mensal_padrao_obs_sim.csv

4) CSV INTERANUAL (OBS + ENSEMBLE + GCM) — para INSETS (violino/box interanual):
   SAZONALIDADE_boxplot_interanual_obs_ensemble.csv  (mantido o nome por compatibilidade)

5) NOVO — SÉRIE MENSAL COMPLETA (OBS + ENSEMBLE + GCM), análogo ao consolidado da FDC:
   SAZONALIDADE_mensal_serie_consolidada.csv
   por_estacao_mensal_serie/SAZONALIDADE_SERIE_MENSAL_<estacao_obs>.csv

Formatos:
A) Climatologia (longo):
   estacao_obs;codigo_mini;horizonte;fonte;modelo;mes;Q_medio

B) Padrão wide:
   estacao_obs;mes;q_obs_ref_mensal;q_sim_mensal

C) Interanual (ano, mes):
   estacao_obs;codigo_mini;horizonte;fonte;modelo;ano;mes;Q_mensal

D) Série mensal completa (ano, mes):
   estacao_obs;codigo_mini;horizonte;fonte;modelo;ano;mes;Q_mensal

onde:
- fonte = OBS | GCM | ENSEMBLE
- modelo = nome do GCM (pasta) | ENSEMBLE_MEAN | OBS
===============================================================================
"""

from __future__ import annotations

from pathlib import Path
from typing import Dict, List, Tuple
import numpy as np
import pandas as pd


# =============================================================================
# CONFIG
# =============================================================================
RAIZ_PROJECOES = Path(r"E:\IGUAÇU_OTTO\7_Projeções_ssp585 - Clima")
OBS_DIR = Path(r"E:\IGUAÇU_OTTO\3_Estações FLU\Input\ascii_mgb")
MAPEAMENTO_CSV = Path(r"E:\IGUAÇU_OTTO\3_Estações FLU\Estações_mini.csv")

OUT_DIR = Path(r"E:\RESULTADOS_AB2\SSP5-85\SAZONALIDADE")
OUT_DIR_EST = OUT_DIR / "por_estacao"
OUT_CSV_ALL = OUT_DIR / "SAZONALIDADE_mensal_consolidada.csv"

# Saída padronizada (wide)
OUT_CSV_PADRAO = OUT_DIR / "SAZONALIDADE_mensal_padrao_obs_sim.csv"

# Saída interanual (AGORA: OBS + ENSEMBLE + GCM)
OUT_CSV_BOX = OUT_DIR / "SAZONALIDADE_boxplot_interanual_obs_ensemble.csv"

# NOVO: série mensal completa (OBS + ENSEMBLE + GCM) — análogo ao consolidado da FDC
OUT_CSV_MENSAL_SERIE = OUT_DIR / "SAZONALIDADE_mensal_serie_consolidada.csv"
OUT_DIR_EST_SERIE = OUT_DIR / "por_estacao_mensal_serie"

SEP = ";"
ENC = "utf-8"

SHIFT_ANOS = 80

# Observado (referência)
REF_OBS = ("1980-01-01", "2023-12-31")

# Horizontes (4)
HORIZONTES: Dict[str, Tuple[str, str]] = {
    "CURTO_2015_2040": ("2015-01-01", "2040-12-31"),
    "MEDIO_2041_2070": ("2041-01-01", "2070-12-31"),
    "LONGO_2071_2100": ("2071-01-01", "2100-12-31"),
    "TOTAL_2015_2100": ("2015-01-01", "2100-12-31"),
}

# =============================================================================
# CSV PADRÃO (wide): qual simulado entra na coluna q_sim_mensal?
# (independente do interanual e da série mensal)
# =============================================================================
PADRAO_HORIZ_OBS = "REF_1980_2023"
PADRAO_FONTE_OBS = "OBS"
PADRAO_MODELO_OBS = "OBS"

PADRAO_HORIZ_SIM = "TOTAL_2015_2100"   # escolha um dos HORIZONTES
PADRAO_FONTE_SIM = "ENSEMBLE"          # "ENSEMBLE" (recomendado) ou "GCM"
PADRAO_MODELO_SIM = "ENSEMBLE_MEAN"    # se fonte=GCM, coloque o nome exato da pasta do modelo


# =============================================================================
# FUNÇÕES
# =============================================================================
def garantir_pasta(p: Path) -> None:
    p.mkdir(parents=True, exist_ok=True)

def listar_modelos(raiz: Path) -> List[Path]:
    modelos = []
    for p in sorted(raiz.iterdir()):
        if p.is_dir() and (p / "Output").exists():
            modelos.append(p)
    return modelos

def ler_txt_mgb(caminho: Path) -> pd.Series:
    df = pd.read_csv(
        caminho, sep=r"\s+", header=None,
        names=["dia", "mes", "ano", "valor"],
        dtype=str, engine="python"
    )
    df["valor"] = df["valor"].str.replace(",", "", regex=False).astype(float)
    df.loc[df["valor"] == -1, "valor"] = np.nan

    df["data"] = pd.to_datetime(
        df.rename(columns={"ano": "year", "mes": "month", "dia": "day"})[["year", "month", "day"]],
        errors="coerce"
    )
    df = df.dropna(subset=["data"]).set_index("data").sort_index()
    return df["valor"]

def aplicar_shift_anos(s: pd.Series, shift_anos: int) -> pd.Series:
    s2 = s.copy()
    s2.index = s2.index + pd.DateOffset(years=shift_anos)
    return s2

def recortar(s: pd.Series, ini: str, fim: str) -> pd.Series:
    return s.loc[(s.index >= pd.Timestamp(ini)) & (s.index <= pd.Timestamp(fim))]

def climatologia_mensal(s: pd.Series) -> pd.Series:
    """
    Climatologia mensal: média de TODOS os dias por mês (1..12) no intervalo.
    """
    x = s.dropna()
    if x.empty:
        return pd.Series(index=range(1, 13), dtype=float)

    df = x.to_frame("q")
    df["mes"] = df.index.month
    clim = df.groupby("mes")["q"].mean().reindex(range(1, 13))
    return clim

def ensemble_diario(series_por_modelo: Dict[str, pd.Series]) -> pd.Series:
    if not series_por_modelo:
        return pd.Series(dtype=float)
    df = pd.concat(series_por_modelo, axis=1)
    ens = df.mean(axis=1, skipna=True)
    ens.name = "ENSEMBLE_MEAN"
    return ens

# -----------------------------------------------------------------------------
# Médias mensais por ANO (para boxplot/violino interanual)
# -----------------------------------------------------------------------------
def medias_mensais_por_ano(s: pd.Series) -> pd.DataFrame:
    """
    Retorna médias mensais por ano.
    Saída: colunas [ano, mes, Q_mensal]
    """
    x = s.dropna()
    if x.empty:
        return pd.DataFrame(columns=["ano", "mes", "Q_mensal"])

    df = x.to_frame("q")
    df["ano"] = df.index.year
    df["mes"] = df.index.month

    out = (
        df.groupby(["ano", "mes"])["q"]
          .mean()
          .reset_index()
          .rename(columns={"q": "Q_mensal"})
    )
    return out

# -----------------------------------------------------------------------------
# NOVO: Série mensal completa (um valor por ano-mês) — análogo ao que você quer
# -----------------------------------------------------------------------------
def serie_mensal(s: pd.Series) -> pd.DataFrame:
    """
    Série mensal: média de todos os dias em cada mês (um valor por ano-mês).
    Saída: colunas [ano, mes, Q_mensal]
    """
    x = s.dropna()
    if x.empty:
        return pd.DataFrame(columns=["ano", "mes", "Q_mensal"])

    sm = x.resample("MS").mean()  # 1 valor por mês (início do mês)
    out = sm.to_frame("Q_mensal").reset_index().rename(columns={"index": "data"})
    # em alguns pandas, reset_index cria coluna com o nome do índice ("data") ou "index"
    if "data" not in out.columns:
        out = out.rename(columns={"index": "data"})
    out["ano"] = out["data"].dt.year.astype(int)
    out["mes"] = out["data"].dt.month.astype(int)
    out = out.drop(columns=["data"])
    return out[["ano", "mes", "Q_mensal"]]


# =============================================================================
# CSV padrão wide (Obs vs Sim)
# =============================================================================
def gerar_csv_padrao(df_all: pd.DataFrame) -> pd.DataFrame:
    """
    Entrada: df_all no formato longo:
      estacao_obs;codigo_mini;horizonte;fonte;modelo;mes;Q_medio

    Saída: formato wide:
      estacao_obs;mes;q_obs_ref_mensal;q_sim_mensal
    """
    df = df_all.copy()

    df["estacao_obs"] = df["estacao_obs"].astype(str).str.strip()
    df["mes"] = pd.to_numeric(df["mes"], errors="coerce").astype("Int64")
    df["Q_medio"] = pd.to_numeric(df["Q_medio"], errors="coerce")

    obs = df[
        (df["horizonte"] == PADRAO_HORIZ_OBS) &
        (df["fonte"] == PADRAO_FONTE_OBS) &
        (df["modelo"] == PADRAO_MODELO_OBS)
    ][["estacao_obs", "mes", "Q_medio"]].rename(columns={"Q_medio": "q_obs_ref_mensal"})

    sim = df[
        (df["horizonte"] == PADRAO_HORIZ_SIM) &
        (df["fonte"] == PADRAO_FONTE_SIM) &
        (df["modelo"] == PADRAO_MODELO_SIM)
    ][["estacao_obs", "mes", "Q_medio"]].rename(columns={"Q_medio": "q_sim_mensal"})

    out = pd.merge(obs, sim, on=["estacao_obs", "mes"], how="outer")

    ests = sorted(out["estacao_obs"].dropna().unique())
    grid = pd.MultiIndex.from_product([ests, range(1, 13)], names=["estacao_obs", "mes"]).to_frame(index=False)
    out = pd.merge(grid, out, on=["estacao_obs", "mes"], how="left")

    return out.sort_values(["estacao_obs", "mes"]).reset_index(drop=True)


# =============================================================================
# MAIN
# =============================================================================
def main():
    garantir_pasta(OUT_DIR)
    garantir_pasta(OUT_DIR_EST)
    garantir_pasta(OUT_DIR_EST_SERIE)

    map_df = pd.read_csv(MAPEAMENTO_CSV, sep=SEP, dtype=str).fillna("")
    modelos = listar_modelos(RAIZ_PROJECOES)
    if not modelos:
        raise FileNotFoundError(f"Nenhuma pasta de modelo com Output/ encontrada em {RAIZ_PROJECOES}")

    rows_all: List[dict] = []           # climatologia (longo)
    rows_box: List[dict] = []           # interanual (ano, mes) p/ violin/box
    rows_mensal_serie: List[dict] = []  # NOVO: série mensal completa (ano, mes)

    for _, row in map_df.iterrows():
        estacao = row.get("estacao_obs", "").strip()
        mini = row.get("codigo_mini", "").strip()
        if not estacao or not mini:
            continue

        rows_est: List[dict] = []        # climatologia por estação
        rows_est_serie: List[dict] = []  # série mensal por estação

        # ---- Observado REF ----
        arq_obs = OBS_DIR / f"{estacao}.txt"
        if arq_obs.exists():
            s_obs = ler_txt_mgb(arq_obs)

            # climatologia mensal OBS (REF)
            clim_obs = climatologia_mensal(recortar(s_obs, *REF_OBS))
            for mes, val in clim_obs.items():
                rows_est.append({
                    "estacao_obs": estacao,
                    "codigo_mini": mini,
                    "horizonte": "REF_1980_2023",
                    "fonte": "OBS",
                    "modelo": "OBS",
                    "mes": int(mes),
                    "Q_medio": float(val) if pd.notna(val) else np.nan,
                })

            # interanual OBS (médias mensais por ano)
            df_obs_box = medias_mensais_por_ano(recortar(s_obs, *REF_OBS))
            for _, rr in df_obs_box.iterrows():
                rows_box.append({
                    "estacao_obs": estacao,
                    "codigo_mini": mini,
                    "horizonte": "REF_1980_2023",
                    "fonte": "OBS",
                    "modelo": "OBS",
                    "ano": int(rr["ano"]),
                    "mes": int(rr["mes"]),
                    "Q_mensal": float(rr["Q_mensal"]) if pd.notna(rr["Q_mensal"]) else np.nan,
                })

            # NOVO: série mensal OBS (REF) — um valor por ano-mês
            df_obs_serie = serie_mensal(recortar(s_obs, *REF_OBS))
            for _, rr in df_obs_serie.iterrows():
                d = {
                    "estacao_obs": estacao,
                    "codigo_mini": mini,
                    "horizonte": "REF_1980_2023",
                    "fonte": "OBS",
                    "modelo": "OBS",
                    "ano": int(rr["ano"]),
                    "mes": int(rr["mes"]),
                    "Q_mensal": float(rr["Q_mensal"]) if pd.notna(rr["Q_mensal"]) else np.nan,
                }
                rows_mensal_serie.append(d)
                rows_est_serie.append(d)

        else:
            print(f"⚠️ {estacao}: observado não encontrado ({arq_obs.name}).")

        # ---- Simulados por modelo + ensemble ----
        sims: Dict[str, pd.Series] = {}
        for pasta_modelo in modelos:
            mname = pasta_modelo.name
            arq_sim = pasta_modelo / "Output" / f"SIM_MC_{mini}.TXT"
            if not arq_sim.exists():
                continue
            s = aplicar_shift_anos(ler_txt_mgb(arq_sim), SHIFT_ANOS)
            sims[mname] = s

        if not sims:
            print(f"⚠️ {estacao}: nenhum simulado encontrado (SIM_MC_{mini}).")
            continue

        # 1) Por modelo — climatologia mensal por horizonte
        for mname, s in sims.items():
            for horiz, (ini, fim) in HORIZONTES.items():
                clim = climatologia_mensal(recortar(s, ini, fim))
                for mes, val in clim.items():
                    rows_est.append({
                        "estacao_obs": estacao,
                        "codigo_mini": mini,
                        "horizonte": horiz,
                        "fonte": "GCM",
                        "modelo": mname,
                        "mes": int(mes),
                        "Q_medio": float(val) if pd.notna(val) else np.nan,
                    })

        # 2) Ensemble diário -> climatologia mensal
        ens = ensemble_diario(sims)
        for horiz, (ini, fim) in HORIZONTES.items():
            clim = climatologia_mensal(recortar(ens, ini, fim))
            for mes, val in clim.items():
                rows_est.append({
                    "estacao_obs": estacao,
                    "codigo_mini": mini,
                    "horizonte": horiz,
                    "fonte": "ENSEMBLE",
                    "modelo": "ENSEMBLE_MEAN",
                    "mes": int(mes),
                    "Q_medio": float(val) if pd.notna(val) else np.nan,
                })

        # 3) Interanual por GCM (ano, mes) por horizonte
        for mname, s in sims.items():
            for horiz, (ini, fim) in HORIZONTES.items():
                df_gcm_box = medias_mensais_por_ano(recortar(s, ini, fim))
                for _, rr in df_gcm_box.iterrows():
                    rows_box.append({
                        "estacao_obs": estacao,
                        "codigo_mini": mini,
                        "horizonte": horiz,
                        "fonte": "GCM",
                        "modelo": mname,
                        "ano": int(rr["ano"]),
                        "mes": int(rr["mes"]),
                        "Q_mensal": float(rr["Q_mensal"]) if pd.notna(rr["Q_mensal"]) else np.nan,
                    })

        # 4) Interanual ENSEMBLE (ano, mes) por horizonte
        for horiz, (ini, fim) in HORIZONTES.items():
            df_ens_box = medias_mensais_por_ano(recortar(ens, ini, fim))
            for _, rr in df_ens_box.iterrows():
                rows_box.append({
                    "estacao_obs": estacao,
                    "codigo_mini": mini,
                    "horizonte": horiz,
                    "fonte": "ENSEMBLE",
                    "modelo": "ENSEMBLE_MEAN",
                    "ano": int(rr["ano"]),
                    "mes": int(rr["mes"]),
                    "Q_mensal": float(rr["Q_mensal"]) if pd.notna(rr["Q_mensal"]) else np.nan,
                })

        # 5) NOVO: Série mensal completa por GCM (ano, mes) por horizonte
        for mname, s in sims.items():
            for horiz, (ini, fim) in HORIZONTES.items():
                df_gcm_serie = serie_mensal(recortar(s, ini, fim))
                for _, rr in df_gcm_serie.iterrows():
                    d = {
                        "estacao_obs": estacao,
                        "codigo_mini": mini,
                        "horizonte": horiz,
                        "fonte": "GCM",
                        "modelo": mname,
                        "ano": int(rr["ano"]),
                        "mes": int(rr["mes"]),
                        "Q_mensal": float(rr["Q_mensal"]) if pd.notna(rr["Q_mensal"]) else np.nan,
                    }
                    rows_mensal_serie.append(d)
                    rows_est_serie.append(d)

        # 6) NOVO: Série mensal completa ENSEMBLE (ano, mes) por horizonte
        for horiz, (ini, fim) in HORIZONTES.items():
            df_ens_serie = serie_mensal(recortar(ens, ini, fim))
            for _, rr in df_ens_serie.iterrows():
                d = {
                    "estacao_obs": estacao,
                    "codigo_mini": mini,
                    "horizonte": horiz,
                    "fonte": "ENSEMBLE",
                    "modelo": "ENSEMBLE_MEAN",
                    "ano": int(rr["ano"]),
                    "mes": int(rr["mes"]),
                    "Q_mensal": float(rr["Q_mensal"]) if pd.notna(rr["Q_mensal"]) else np.nan,
                }
                rows_mensal_serie.append(d)
                rows_est_serie.append(d)

        # ---- salva por estação (climatologia) ----
        df_est = pd.DataFrame(rows_est)
        out_est = OUT_DIR_EST / f"SAZONALIDADE_{estacao}.csv"
        df_est.to_csv(out_est, sep=SEP, index=False, encoding=ENC)

        # ---- salva por estação (série mensal completa) ----
        df_est_serie = pd.DataFrame(rows_est_serie)
        out_est_serie = OUT_DIR_EST_SERIE / f"SAZONALIDADE_SERIE_MENSAL_{estacao}.csv"
        df_est_serie.to_csv(out_est_serie, sep=SEP, index=False, encoding=ENC)

        rows_all.extend(rows_est)
        print(f"✅ OK: {estacao} | modelos={len(sims)} | salvo={out_est.name} | serie={out_est_serie.name}")

    # ---- consolidado climatologia (longo) ----
    df_all = pd.DataFrame(rows_all)
    df_all.to_csv(OUT_CSV_ALL, sep=SEP, index=False, encoding=ENC)
    print(f"\n✅ Consolidado (climatologia) salvo: {OUT_CSV_ALL}")

    # ---- padronizado (wide) ----
    df_padrao = gerar_csv_padrao(df_all)
    df_padrao.to_csv(OUT_CSV_PADRAO, sep=SEP, index=False, encoding=ENC)
    print(f"✅ Padrão (wide) salvo: {OUT_CSV_PADRAO}")

    # ---- interanual (OBS + ENSEMBLE + GCM) ----
    df_box = pd.DataFrame(rows_box)
    df_box.to_csv(OUT_CSV_BOX, sep=SEP, index=False, encoding=ENC)
    print(f"✅ Interanual (OBS + ENSEMBLE + GCM) salvo: {OUT_CSV_BOX}")

    # ---- NOVO: série mensal completa (OBS + ENSEMBLE + GCM) ----
    df_serie = pd.DataFrame(rows_mensal_serie)
    df_serie.to_csv(OUT_CSV_MENSAL_SERIE, sep=SEP, index=False, encoding=ENC)
    print(f"✅ Série mensal completa (OBS + ENSEMBLE + GCM) salva: {OUT_CSV_MENSAL_SERIE}")

    # DIAG opcional
    try:
        fontes = sorted(df_box["fonte"].dropna().unique())
        print("\n[DIAG] Interanual: fontes únicas =", fontes)
        if "GCM" in fontes:
            ngcm = df_box.loc[df_box["fonte"] == "GCM", "modelo"].nunique()
            print(f"[DIAG] Interanual: #GCMs distintos = {ngcm}")
    except Exception:
        pass

    try:
        fontes2 = sorted(df_serie["fonte"].dropna().unique())
        print("[DIAG] Série mensal: fontes únicas =", fontes2)
        if "GCM" in fontes2:
            ngcm2 = df_serie.loc[df_serie["fonte"] == "GCM", "modelo"].nunique()
            print(f"[DIAG] Série mensal: #GCMs distintos = {ngcm2}")
    except Exception:
        pass

    print("\n[INFO] CSV padrão (wide) usa:")
    print(f"  OBS: horizonte={PADRAO_HORIZ_OBS} | fonte={PADRAO_FONTE_OBS} | modelo={PADRAO_MODELO_OBS}")
    print(f"  SIM: horizonte={PADRAO_HORIZ_SIM} | fonte={PADRAO_FONTE_SIM} | modelo={PADRAO_MODELO_SIM}")

    print("\n[INFO] CSV interanual (boxplot/violin) contém:")
    print("  OBS: REF_1980_2023 | fonte=OBS | modelo=OBS | Q_mensal por (ano, mes)")
    print("  GCM: cada horizonte | fonte=GCM | modelo=<nome do GCM> | Q_mensal por (ano, mes)")
    print("  ENSEMBLE: cada horizonte | fonte=ENSEMBLE | modelo=ENSEMBLE_MEAN | Q_mensal por (ano, mes)")

    print("\n[INFO] CSV série mensal completa contém:")
    print("  OBS: REF_1980_2023 | fonte=OBS | modelo=OBS | Q_mensal por (ano, mes)")
    print("  GCM: cada horizonte | fonte=GCM | modelo=<nome do GCM> | Q_mensal por (ano, mes)")
    print("  ENSEMBLE: cada horizonte | fonte=ENSEMBLE | modelo=ENSEMBLE_MEAN | Q_mensal por (ano, mes)")


if __name__ == "__main__":
    main()

In [ ]:
# -*- coding: utf-8 -*-
"""
===============================================================================
SCRIPT 01 — BASE CARTOGRÁFICA CONSOLIDADA (SEM RECORTE / SEM FILTRO)

Premissas
---------
- Hidrografia já está:
  • recortada ao limite da bacia/sub-bacias
  • filtrada por ordem (ex.: <= 5)
- Este script NÃO altera geometria nem conteúdo temático.
- Apenas organiza, padroniza CRS, rotula sub-bacias e salva uma base única.

Saídas
------
- final_base.gpkg  -> camadas prontas para plotagem
- base_meta.json   -> metadados para uso no Script 02
===============================================================================
"""

from __future__ import annotations

import json
from pathlib import Path
from typing import Optional, Dict, Any

import geopandas as gpd
import pandas as pd


# =============================================================================
# CAMINHOS (GPKG por camada)
# =============================================================================
gpkg_cidades   = Path(r"E:\Base_Cartografica_GPKG\cidades.gpkg")
gpkg_estacoes  = Path(r"E:\Base_Cartografica_GPKG\estacoes.gpkg")
gpkg_hidro     = Path(r"E:\Base_Cartografica_GPKG\hidrografia.gpkg")
gpkg_minis     = Path(r"E:\Base_Cartografica_GPKG\minis_mgb.gpkg")
gpkg_subbacias = Path(r"E:\Base_Cartografica_GPKG\subbacias.gpkg")

LAYER_CIDADES   = "cidades"
LAYER_ESTACOES  = "estacoes"
LAYER_HIDRO     = "hidrografia"
LAYER_MINIS     = "minis_mgb"
LAYER_SUBBACIAS = "subbacias"

# Campos
SUB_ID_FIELD = "Sub_Basin"            # ex.: "SB" se existir
CAMPO_NOME_CIDADE = "CID_NM"

# CRS alvo
TARGET_EPSG = 31982

# Filtro simples de cidades (opcional)
CIDADES_SELECIONADAS = ["Curitiba", "União da Vitória"]

# Saída
OUT_DIR = Path(r"E:\Base_Cartografica_GPKG\_base")
OUT_DIR.mkdir(parents=True, exist_ok=True)

OUT_GPKG = OUT_DIR / "final_base.gpkg"
OUT_META = OUT_DIR / "base_meta.json"


# =============================================================================
# FUNÇÕES AUXILIARES
# =============================================================================

def ensure_crs(gdf: gpd.GeoDataFrame, target_epsg: int) -> gpd.GeoDataFrame:
    if gdf.crs is None:
        raise ValueError("Camada sem CRS definido.")
    if gdf.crs.to_epsg() != target_epsg:
        return gdf.to_crs(epsg=target_epsg)
    return gdf

def label_subbasins(
    gdf_sub: gpd.GeoDataFrame,
    field: Optional[str]
) -> gpd.GeoDataFrame:

    gdf = gdf_sub.copy()

    if field is None:
        raise ValueError(      "SUB_ID_FIELD está None. Defina, por exemplo: SUB_ID_FIELD = 'Sub_Basin'."  )

    if field not in gdf.columns:
        raise KeyError(  f"Campo '{field}' não existe em subbacias. "
            f"Colunas disponíveis: {list(gdf.columns)}"
        )

    # extrai apenas o número da sub-bacia
    s = gdf[field].astype(str).str.strip()
    num = s.str.replace(r"\D", "", regex=True)

    # rótulo final padronizado
    gdf["SB_LBL"] = "SB-" + num.str.zfill(2)

    return gdf

def detectar_campo_codigo_estacao(gdf_est: gpd.GeoDataFrame) -> str:
    cols = list(gdf_est.columns)
    lower_map = {c.lower(): c for c in cols}

    candidatos = [
        "codigo", "cod", "code", "station", "estacao",
        "id", "id_est", "cd_estacao"
    ]
    for c in candidatos:
        if c.lower() in lower_map:
            return lower_map[c.lower()]

    # heurística: coluna com muitos números longos
    best = None
    best_score = -1
    for c in cols:
        if c == gdf_est.geometry.name:
            continue
        s = gdf_est[c].astype(str).str.replace(r"\D", "", regex=True)
        score = (s.str.len() >= 6).sum()
        if score > best_score:
            best_score = score
            best = c

    if best is None:
        raise KeyError("Não foi possível detectar o campo do código da estação.")
    return best


# =============================================================================
# MAIN
# =============================================================================

def main() -> None:
    # -------------------------------------------------------------------------
    # 1) Leitura das camadas
    # -------------------------------------------------------------------------
    gdf_minis = gpd.read_file(gpkg_minis, layer=LAYER_MINIS)
    gdf_sub   = gpd.read_file(gpkg_subbacias, layer=LAYER_SUBBACIAS)
    gdf_hidro = gpd.read_file(gpkg_hidro, layer=LAYER_HIDRO)
    gdf_cid   = gpd.read_file(gpkg_cidades, layer=LAYER_CIDADES)
    gdf_est   = gpd.read_file(gpkg_estacoes, layer=LAYER_ESTACOES)

    # -------------------------------------------------------------------------
    # 2) CRS
    # -------------------------------------------------------------------------
    gdf_minis = ensure_crs(gdf_minis, TARGET_EPSG)
    gdf_sub   = ensure_crs(gdf_sub, TARGET_EPSG)
    gdf_hidro = ensure_crs(gdf_hidro, TARGET_EPSG)
    gdf_cid   = ensure_crs(gdf_cid, TARGET_EPSG)
    gdf_est   = ensure_crs(gdf_est, TARGET_EPSG)

    # -------------------------------------------------------------------------
    # 3) Sub-bacias rotuladas
    # -------------------------------------------------------------------------
    gdf_sub = label_subbasins(gdf_sub, SUB_ID_FIELD)

    # -------------------------------------------------------------------------
    # 4) Cidades (filtro simples)
    # -------------------------------------------------------------------------
    if CAMPO_NOME_CIDADE in gdf_cid.columns:
        gdf_cid = gdf_cid[gdf_cid[CAMPO_NOME_CIDADE].isin(CIDADES_SELECIONADAS)].copy()
    else:
        print(f"[AVISO] Campo {CAMPO_NOME_CIDADE} não encontrado em cidades.")

    # -------------------------------------------------------------------------
    # 5) Estações: detectar campo de código e normalizar
    # -------------------------------------------------------------------------
    campo_cod_est = detectar_campo_codigo_estacao(gdf_est)

    gdf_est = gdf_est.copy()
    gdf_est["_cod_str"] = (
        gdf_est[campo_cod_est]
        .astype(str)
        .str.strip()
        .str.replace(r"\.0$", "", regex=True)
    )

    # -------------------------------------------------------------------------
    # 6) Exportar GPKG consolidado (sem alterar geometrias)
    # -------------------------------------------------------------------------
    if OUT_GPKG.exists():
        OUT_GPKG.unlink()

    gdf_minis.to_file(OUT_GPKG, layer="minis", driver="GPKG")
    gdf_sub.to_file(OUT_GPKG, layer="subbacias", driver="GPKG")
    gdf_hidro.to_file(OUT_GPKG, layer="hidrografia", driver="GPKG")
    gdf_cid.to_file(OUT_GPKG, layer="cidades", driver="GPKG")
    gdf_est.to_file(OUT_GPKG, layer="estacoes", driver="GPKG")

    # -------------------------------------------------------------------------
    # 7) Metadados (para Script 02)
    # -------------------------------------------------------------------------
    bounds = gdf_sub.total_bounds
    basin_center = gdf_sub.unary_union.centroid

    meta: Dict[str, Any] = {
        "target_epsg": TARGET_EPSG,
        "station_code_aux_field": "_cod_str",
        "station_code_original_field": campo_cod_est,
        "sub_label_field": "SB_LBL",
        "sub_id_field_original": SUB_ID_FIELD,
        "map_bounds": {
            "xmin": float(bounds[0]),
            "ymin": float(bounds[1]),
            "xmax": float(bounds[2]),
            "ymax": float(bounds[3]),
        },
        "basin_center_xy": {
            "x": float(basin_center.x),
            "y": float(basin_center.y),
        },
        "layers": {
            "minis": "minis",
            "subbacias": "subbacias",
            "hidrografia": "hidrografia",
            "cidades": "cidades",
            "estacoes": "estacoes",
        },
    }

    OUT_META.write_text(
        json.dumps(meta, ensure_ascii=False, indent=2),
        encoding="utf-8",
    )

    print("✅ Base cartográfica consolidada com sucesso:")
    print(f"   GPKG : {OUT_GPKG}")
    print(f"   META : {OUT_META}")
    print(f"   Campo código estação: {campo_cod_est}")


if __name__ == "__main__":
    main()

In [ ]:
#RODA MAPAS DE FORMA MANUAL
# -*- coding: utf-8 -*-
"""
===============================================================================
SCRIPT 02 — MAPA FINAL + INSETS (FDC ou SAZONALIDADE) — LAYOUT FIXO (manual)

- Mantém minis + sub-bacias + simbologia da hidrografia por ordem
- Insets com posições FIXAS (slots) conforme layout desenhado (INSET_POS)
- Callouts conectam a estação ao canto mais próximo do inset

MODOS DISPONÍVEIS
1) FDC:
   - Curva de permanência (opcional log)
2) SAZONALIDADE — BOX (GCMs):
   - Boxplots mensais (amostra = 19 GCMs) + linha Obs + linha Ensemble
3) SAZONALIDADE — VIOLIN/BOX INTERANUAL (ENSEMBLE):
   - Violino+box mensal do ENSEMBLE (amostra = anos, usando Q_mensal)
   - Linhas suaves: mediana mensal do ENSEMBLE + mediana mensal do OBS
   - Violinos coloridos (1 cor por mês)
   - Meses no eixo X de cada inset (sem legenda)

APÊNDICE — FIGURAS INDIVIDUAIS POR MODELO
- FDC: gera também 1 mapa por GCM (além do MODELO_PLOT), usando o mesmo layout.
- SAZONALIDADE (VIOLIN/BOX interanual): gera também 1 mapa por modelo GCM (fonte="GCM"),
  mantendo OBS como referência.
- SAZONALIDADE (BOX GCMs): por definição NÃO gera por modelo (box = distribuição intermodelo).

OBS: Nenhuma alteração de estética/edição gráfica foi feita — apenas unificação + loop por modelo.
===============================================================================
"""

from __future__ import annotations

import json
from pathlib import Path
from typing import Dict, Tuple, Optional, List

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

# usado apenas no modo VIOLIN (mantém igual ao seu script 2)
import matplotlib as mpl


# =============================================================================
# ENTRADAS: BASE (saída do Script 01)
# =============================================================================
BASE_DIR = Path(r"E:\Base_Cartografica_GPKG\_base")
BASE_GPKG = BASE_DIR / "final_base.gpkg"
BASE_META = BASE_DIR / "base_meta.json"

# Saída
OUT_DIR = Path(r"E:\RESULTADOS\SSP5_85\Figuras_FDC")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Subpasta p/ apêndice (mapas individuais por modelo)
OUT_APPENDIX_DIR = OUT_DIR / "_appendice_modelos"
OUT_APPENDIX_DIR.mkdir(parents=True, exist_ok=True)


# =============================================================================
# ESCOLHA DO TIPO DE INSET
#   - "fdc"
#   - "seasonal_box_gcms"            (BOX: 19 GCMs + Obs + Ensemble)
#   - "seasonal_violin_interannual"  (VIOLIN/BOX: Ensemble interanual + linhas)
# =============================================================================
PLOT_KIND = "fdc"

# =============================================================================
# APÊNDICE: GERAR MAPAS INDIVIDUAIS POR MODELO?
# - FDC: itera sobre os modelos (GCMs) existentes no CSV
# - VIOLIN interanual: itera sobre MODELO_COL dentro de fonte="GCM"
# - seasonal_box_gcms: não gera por modelo (não é metodologicamente adequado)
# =============================================================================
GENERATE_APPENDIX_PER_MODEL = True


# =============================================================================
# LAYOUT FIGURA
# =============================================================================
FIGSIZE = (11.69, 8.27)  # A4 paisagem
DPI = 300
AX_RECT = [0.05, 0.08, 0.90, 0.84]

# Tamanho dos insets
INSET_W = 0.21
INSET_H = 0.18


# =============================================================================
# FDC
# =============================================================================
CSV_FDC = Path(r"E:\RESULTADOS\SSP2_45\FDC_Projecoes_ssp245\FDC_consolidado_obs_ref_e_futuro.csv")
FDC_SEP = ";"
HORIZONTE_PLOT = "TOTAL_2015_2100"
MODELO_PLOT = "ENSEMBLE_MEAN_POS"
EXC_COL = "exc"
QSIM_COL = "q_sim"
QOBS_COL = "q_obs_ref"
FDC_YLOG = True  # log no eixo Y da FDC


# =============================================================================
# SAZONALIDADE (BOX: 19 GCMs) — CSV CONSOLIDADO (formato longo)
# Colunas esperadas:
# estacao_obs;codigo_mini;horizonte;fonte;modelo;mes;Q_medio
# =============================================================================
CSV_SEASON = Path(r"E:\RESULTADOS\SSP2_45\Sazonalidade\SAZONALIDADE_mensal_consolidada.csv")
SEASON_SEP = ";"

EST_COL   = "estacao_obs"
HORIZ_COL = "horizonte"
FONTE_COL = "fonte"
MODELO_COL = "modelo"
MES_COL   = "mes"
Q_COL     = "Q_medio"

HORIZONTE_SIM = "TOTAL_2015_2100"   # CURTO_2015_2040 / MEDIO_2041_2070 / LONGO_2071_2100 / TOTAL_2015_2100
HORIZONTE_OBS = "REF_1980_2023"

SHOW_OBS = True
SHOW_ENSEMBLE = True
SEASON_YLOG = False  # se quiser sazonal em log, mude p/ True


# =============================================================================
# SAZONALIDADE — VIOLIN/BOX INTERANUAL (OBS + ENSEMBLE)
# CSV:
# estacao_obs;codigo_mini;horizonte;fonte;modelo;ano;mes;Q_mensal
# =============================================================================
CSV_SEASON_BOX = Path(r"E:\RESULTADOS\SSP2_45\SAZONALIDADE\SAZONALIDADE_boxplot_interanual_obs_ensemble.csv")

ANO_COL = "ano"
Q_COL_BOX = "Q_mensal"

# Paleta qualitativa (tab20 existe; usamos 12 cores)
MONTH_CMAP = mpl.colormaps["tab20"]
MONTH_LABELS = ["Jan","Fev","Mar","Abr","Mai","Jun","Jul","Ago","Set","Out","Nov","Dez"]


# =============================================================================
# POSIÇÕES FIXAS (slots) — coordenadas da FIGURA (0–1)
# =============================================================================
INSET_POS: Dict[str, Tuple[float, float]] = {
    # topo
    "65208000": (0.15, 0.86),
    "65060000": (0.52, 0.86),
    "65035000": (0.82, 0.86),

    # lados
    "65310000": (0.08, 0.55),
    "65155000": (0.82, 0.55),

    # base
    "65220000": (0.33, 0.15),
    "65295000": (0.08, 0.18),
    "65175000": (0.58, 0.15),
    "65095000": (0.82, 0.28),
}


# =============================================================================
# CARTOGRAFIA (mantém padrão)
# =============================================================================
COLOR_MINIS = "#8a8a8a"
COLOR_SUB_EDGE = "#cc0000"

COLOR_H_13 = "#1f4e79"
COLOR_H_4  = "#4f81bd"
COLOR_H_5  = "#9dc3e6"

LW_MINIS = 0.25
LW_SUB_EDGE = 1.10
LW_H_13 = 0.75
LW_H_4  = 0.95
LW_H_5  = 1.10

ALPHA_SUB_FILL = 0.22

CALL_OUT_COLOR = "crimson"
CALL_OUT_LW = 0.9
CALL_OUT_ALPHA = 0.85


# =============================================================================
# HELPERS (apêndice por modelo)
# =============================================================================
def _sanitize_name(s: str) -> str:
    s = str(s).strip()
    for ch in ["/", "\\", ":", "*", "?", '"', "<", ">", "|"]:
        s = s.replace(ch, "_")
    s = s.replace(" ", "_")
    return s

def _looks_like_ensemble_or_obs(model_name: str) -> bool:
    m = str(model_name).strip().upper()
    if m in {"OBS", "OBS_REF", "OBSERVADO", "OBSERVED"}:
        return True
    if "ENSEMBLE" in m:
        return True
    return False


# =============================================================================
# FUNÇÕES AUXILIARES (comuns)
# =============================================================================
def data_to_figcoords(ax, x, y) -> Tuple[float, float]:
    xy_disp = ax.transData.transform((x, y))
    xy_fig = ax.figure.transFigure.inverted().transform(xy_disp)
    return float(xy_fig[0]), float(xy_fig[1])

def make_inset(fig, center_xy, w, h):
    cx, cy = center_xy
    left = cx - w / 2
    bottom = cy - h / 2
    return fig.add_axes([left, bottom, w, h])

def inset_corners(center_xy, w, h):
    cx, cy = center_xy
    left = cx - w/2
    right = cx + w/2
    bottom = cy - h/2
    top = cy + h/2
    return [(left, bottom), (left, top), (right, bottom), (right, top)]

def nearest_corner_of_inset(center_xy, w, h, target_xy):
    tx, ty = target_xy
    corners = inset_corners(center_xy, w, h)
    return min(corners, key=lambda p: (p[0]-tx)**2 + (p[1]-ty)**2)

def detectar_campo_ordem(gdf_h: gpd.GeoDataFrame) -> Optional[str]:
    cols = list(gdf_h.columns)
    if "_ord" in cols:
        return "_ord"
    lower_map = {c.lower(): c for c in cols}
    candidatos = ["nuordemcda", "nuordemdca", "nuordem", "ordem", "order", "ord", "strahler", "str_order"]
    for c in candidatos:
        if c.lower() in lower_map:
            return lower_map[c.lower()]
    heur = [c for c in cols if ("ord" in c.lower()) or ("strah" in c.lower())]
    return heur[0] if heur else None

def plot_hidrografia(ax, gdf_h: gpd.GeoDataFrame):
    campo = detectar_campo_ordem(gdf_h)
    if campo is None:
        gdf_h.plot(ax=ax, color=COLOR_H_4, linewidth=0.8, alpha=0.9, zorder=3)
        return

    ords = pd.to_numeric(gdf_h[campo], errors="coerce").fillna(-1).astype(int)
    g = gdf_h.assign(_ord2=ords)
    g = g[(g["_ord2"] >= 1) & (g["_ord2"] <= 5)].copy()

    g13 = g[g["_ord2"].isin([1, 2, 3])]
    g4  = g[g["_ord2"] == 4]
    g5  = g[g["_ord2"] == 5]

    if len(g13):
        g13.plot(ax=ax, color=COLOR_H_13, linewidth=LW_H_13, alpha=0.9, zorder=3)
    if len(g4):
        g4.plot(ax=ax, color=COLOR_H_4, linewidth=LW_H_4, alpha=0.7, zorder=3)
    if len(g5):
        g5.plot(ax=ax, color=COLOR_H_5, linewidth=LW_H_5, alpha=0.5, zorder=3)

def plot_estacoes(ax, gdf_est: gpd.GeoDataFrame, code_field: str):
    gdf_est.plot(ax=ax, marker="v", markersize=70, color="black",
                 edgecolor="white", linewidth=0.6, zorder=6)
    for _, r in gdf_est.iterrows():
        txt = str(r.get(code_field, "")).strip()
        if not txt:
            continue
        ax.annotate(
            txt,
            xy=(r.geometry.x, r.geometry.y),
            xytext=(4, 4),
            textcoords="offset points",
            fontsize=8,
            color="black",
            zorder=7,
        )

def plot_cidades(ax, gdf_cid: gpd.GeoDataFrame, nome_field: str):
    if gdf_cid.empty or nome_field not in gdf_cid.columns:
        return
    gdf_cid.plot(ax=ax, marker="o", markersize=35, color="white",
                 edgecolor="black", linewidth=0.8, zorder=7)
    for _, r in gdf_cid.iterrows():
        ax.annotate(
            str(r[nome_field]),
            xy=(r.geometry.x, r.geometry.y),
            xytext=(5, -8),
            textcoords="offset points",
            fontsize=8,
            color="black",
            zorder=8,
        )


# =============================================================================
# INSETS: FDC
# =============================================================================
def plot_fdc_inset(ax_in, df_fdc: pd.DataFrame, title: str):
    df_fdc = df_fdc.sort_values(EXC_COL).copy()

    def pos(s):
        s = pd.to_numeric(s, errors="coerce")
        return s.where(s > 0)

    x = pd.to_numeric(df_fdc[EXC_COL], errors="coerce")
    y_sim = pos(df_fdc[QSIM_COL])

    if QOBS_COL in df_fdc.columns:
        y_obs = pos(df_fdc[QOBS_COL])
        if y_obs.notna().any():
            ax_in.plot(x, y_obs, lw=1.2, color="black", label="Obs")

    ax_in.plot(x, y_sim, lw=1.1, label="Sim")

    ax_in.set_xlim(0, 100)
    if FDC_YLOG:
        ax_in.set_yscale("log")

    ax_in.set_title(title, fontsize=7, pad=2)
    ax_in.set_xlabel("Excedência (%)", fontsize=6)
    ax_in.set_ylabel("Q (m³/s)", fontsize=6)
    ax_in.tick_params(labelsize=6, length=2)
    ax_in.grid(True, alpha=0.22, linewidth=0.5)
    ax_in.legend(fontsize=5.4, loc="best", frameon=True)


# =============================================================================
# INSETS: SAZONALIDADE — BOX (19 GCMs) + Obs + Ensemble
# =============================================================================
def plot_sazonal_box_modelos_inset(
    ax_in: plt.Axes,
    df_season: pd.DataFrame,
    estacao: str,
    horizonte_sim: str,
    horizonte_obs: str = "REF_1980_2023",
    show_obs: bool = True,
    show_ens: bool = True,
    ylog: bool = False
) -> None:
    dfe = df_season[df_season[EST_COL].astype(str) == str(estacao)].copy()
    if dfe.empty:
        return

    dfe[MES_COL] = pd.to_numeric(dfe[MES_COL], errors="coerce")
    dfe[Q_COL] = pd.to_numeric(dfe[Q_COL], errors="coerce")

    # Boxplots: GCMs (distribuição intermodelo)
    gcm = dfe[(dfe[FONTE_COL] == "GCM") & (dfe[HORIZ_COL] == horizonte_sim)].copy()

    boxes = []
    for m in range(1, 13):
        vals = gcm.loc[gcm[MES_COL] == m, Q_COL].dropna().values.astype(float)
        boxes.append(vals)

    ax_in.boxplot(
        boxes,
        positions=np.arange(1, 13),
        widths=0.55,
        showfliers=False
    )

    # Observado (linha)
    if show_obs:
        obs = dfe[
            (dfe[FONTE_COL] == "OBS") &
            (dfe[MODELO_COL] == "OBS") &
            (dfe[HORIZ_COL] == horizonte_obs)
        ].dropna(subset=[MES_COL, Q_COL]).sort_values(MES_COL)

        if not obs.empty:
            ax_in.plot(obs[MES_COL], obs[Q_COL], color="black", lw=1.2, label="Obs")

    # Ensemble mean (linha)
    if show_ens:
        ens = dfe[
            (dfe[FONTE_COL] == "ENSEMBLE") &
            (dfe[MODELO_COL] == "ENSEMBLE_MEAN") &
            (dfe[HORIZ_COL] == horizonte_sim)
        ].dropna(subset=[MES_COL, Q_COL]).sort_values(MES_COL)

        if not ens.empty:
            ax_in.plot(ens[MES_COL], ens[Q_COL], lw=1.2, label="Ensemble")

    ax_in.set_xlim(0.5, 12.5)
    ax_in.set_xticks([1, 3, 5, 7, 9, 11])
    ax_in.set_xlabel("Mês", fontsize=6)
    ax_in.set_ylabel("Q (m³/s)", fontsize=6)
    ax_in.set_title(str(estacao), fontsize=7, pad=2)
    ax_in.tick_params(labelsize=6, length=2)
    ax_in.grid(True, alpha=0.22, linewidth=0.5)

    if ylog:
        ymin = np.inf
        ymax = 0.0
        for arr in boxes:
            if arr.size:
                arrp = arr[arr > 0]
                if arrp.size:
                    ymin = min(ymin, float(arrp.min()))
                    ymax = max(ymax, float(arrp.max()))
        if np.isfinite(ymin) and ymax > 0:
            ax_in.set_yscale("log")
            ax_in.set_ylim(max(ymin, 0.01), ymax * 1.2)

    handles, labels = ax_in.get_legend_handles_labels()
    if labels:
        ax_in.legend(fontsize=5.2, loc="best", frameon=True)


# =============================================================================
# INSETS: SAZONALIDADE — VIOLIN/BOX INTERANUAL (mantido)
# =============================================================================
def plot_sazonal_violin_interanual_inset(
    ax_in: plt.Axes,
    df_box: pd.DataFrame,
    estacao: str,
    horizonte_sim: str,
    horizonte_obs: str,
    sim_fonte: str,
    sim_modelo: str,
    ylog: bool = False
) -> None:
    dfe = df_box[df_box[EST_COL].astype(str) == str(estacao)].copy()
    if dfe.empty:
        return

    dfe[MES_COL] = pd.to_numeric(dfe[MES_COL], errors="coerce")
    dfe[Q_COL_BOX] = pd.to_numeric(dfe[Q_COL_BOX], errors="coerce")

    sim = dfe[
        (dfe[FONTE_COL] == sim_fonte) &
        (dfe[MODELO_COL] == sim_modelo) &
        (dfe[HORIZ_COL] == horizonte_sim)
    ].copy()

    boxes = []
    sim_med = []
    for m in range(1, 13):
        vals = sim.loc[sim[MES_COL] == m, Q_COL_BOX].dropna().values.astype(float)
        boxes.append(vals)
        sim_med.append(np.nanmean(vals) if vals.size else np.nan)
    sim_med = np.array(sim_med, dtype=float)

    x = np.arange(1, 13)

    vparts = ax_in.violinplot(
        boxes,
        positions=x,
        widths=0.85,
        showmeans=False,
        showmedians=False,
        showextrema=False
    )
    for i, body in enumerate(vparts["bodies"]):
        color = MONTH_CMAP(i)
        body.set_facecolor(color)
        body.set_edgecolor("black")
        body.set_alpha(0.55)
        body.set_linewidth(0.6)

    ax_in.boxplot(
        boxes,
        positions=x,
        widths=0.22,
        showfliers=False,
        patch_artist=True,
        boxprops=dict(facecolor="white", edgecolor="black", linewidth=0.9),
        medianprops=dict(color="black", linewidth=1.0),
        whiskerprops=dict(color="black", linewidth=0.9),
        capprops=dict(color="black", linewidth=0.9),
    )

    obs = dfe[
        (dfe[FONTE_COL] == "OBS") &
        (dfe[MODELO_COL] == "OBS") &
        (dfe[HORIZ_COL] == horizonte_obs)
    ].copy()

    obs_med = []
    for m in range(1, 13):
        vals = obs.loc[obs[MES_COL] == m, Q_COL_BOX].dropna().values.astype(float)
        obs_med.append(np.nanmean(vals) if vals.size else np.nan)
    obs_med = np.array(obs_med, dtype=float)

    def _plot_smooth_line(xv, yv, label, color=None, linestyle="-"):
        mask = np.isfinite(yv)
        if mask.sum() < 3:
            ax_in.plot(xv[mask], yv[mask], label=label, color=color, lw=1.2, linestyle=linestyle)
            return
        try:
            from scipy.interpolate import PchipInterpolator
            xs = np.linspace(1, 12, 240)
            f = PchipInterpolator(xv[mask], yv[mask])
            ys = f(xs)
            ax_in.plot(xs, ys, label=label, color=color, lw=1.2, linestyle=linestyle)
        except Exception:
            ax_in.plot(xv[mask], yv[mask], label=label, color=color, lw=1.2, linestyle=linestyle)

    _plot_smooth_line(x, sim_med, label=f"{sim_modelo} (mediana)", color=None, linestyle="-")
    _plot_smooth_line(x, obs_med, label="Obs (mediana)", color="black", linestyle="-")

    ax_in.set_xlim(0.5, 12.5)
    ax_in.set_xticks(np.arange(1, 13))
    ax_in.set_xticklabels(MONTH_LABELS, fontsize=5.2, rotation=35, ha="right")
    ax_in.set_xlabel("")
    ax_in.set_ylabel("Q (m³/s)", fontsize=6)
    ax_in.set_title(str(estacao), fontsize=7, pad=2)
    ax_in.tick_params(axis="y", labelsize=6, length=2)
    ax_in.grid(True, alpha=0.22, linewidth=0.5)

    if ylog:
        ymin = np.inf
        ymax = 0.0
        for arr in boxes:
            if arr.size:
                arrp = arr[arr > 0]
                if arrp.size:
                    ymin = min(ymin, float(arrp.min()))
                    ymax = max(ymax, float(arrp.max()))
        for v in np.r_[sim_med, obs_med]:
            if np.isfinite(v) and v > 0:
                ymin = min(ymin, float(v))
                ymax = max(ymax, float(v))

        if np.isfinite(ymin) and ymax > 0:
            ax_in.set_yscale("log")
            ax_in.set_ylim(max(ymin, 0.01), ymax * 1.2)

    handles, labels = ax_in.get_legend_handles_labels()
    if labels:
        ax_in.legend(fontsize=5.0, loc="best", frameon=True)


# =============================================================================
# FUNÇÃO: desenha e salva 1 mapa (reusa exatamente a mesma estética)
# =============================================================================
def render_map(
    gdf_minis: gpd.GeoDataFrame,
    gdf_sub: gpd.GeoDataFrame,
    gdf_hidro: gpd.GeoDataFrame,
    gdf_cid: gpd.GeoDataFrame,
    gdf_est: gpd.GeoDataFrame,
    code_field: str,
    est_fig: Dict[str, Tuple[float, float]],
    plot_kind: str,
    out_png: Path,
    df_fdc: Optional[pd.DataFrame] = None,
    df_season: Optional[pd.DataFrame] = None,
    df_box: Optional[pd.DataFrame] = None,
    fdc_modelo: Optional[str] = None,
    violin_sim_fonte: Optional[str] = None,
    violin_sim_modelo: Optional[str] = None,
) -> None:
    fig = plt.figure(figsize=FIGSIZE, dpi=DPI)
    ax = fig.add_axes(AX_RECT)

    gdf_minis.boundary.plot(ax=ax, color=COLOR_MINIS, linewidth=LW_MINIS, alpha=0.55, zorder=1)

    if "SB_LBL" in gdf_sub.columns:
        gdf_sub.plot(
            ax=ax,
            column="SB_LBL",
            alpha=ALPHA_SUB_FILL,
            linewidth=LW_SUB_EDGE,
            edgecolor=COLOR_SUB_EDGE,
            zorder=2,
            legend=False
        )
    else:
        gdf_sub.boundary.plot(ax=ax, color=COLOR_SUB_EDGE, linewidth=LW_SUB_EDGE, zorder=2)

    plot_hidrografia(ax, gdf_hidro)
    plot_cidades(ax, gdf_cid, nome_field="CID_NM")
    plot_estacoes(ax, gdf_est, code_field=code_field)

    if "SB_LBL" in gdf_sub.columns:
        gdf_sub_tmp = gdf_sub.copy()
        gdf_sub_tmp["centroid"] = gdf_sub_tmp.geometry.centroid
        for _, r in gdf_sub_tmp.iterrows():
            ax.annotate(
                str(r["SB_LBL"]),
                xy=(r["centroid"].x, r["centroid"].y),
                ha="center", va="center",
                fontsize=10, fontweight="bold",
                color="#7a0000", zorder=10
            )

    ax.set_axis_off()

    for st, center in INSET_POS.items():
        if st not in est_fig:
            continue

        ax_in = make_inset(fig, center, INSET_W, INSET_H)

        if plot_kind == "fdc":
            if df_fdc is None:
                continue
            modelo = fdc_modelo if fdc_modelo is not None else MODELO_PLOT

            df_st = df_fdc[
                (df_fdc["estacao_obs"].astype(str) == str(st)) &
                (df_fdc["modelo"].astype(str) == str(modelo)) &
                (df_fdc["horizonte"].astype(str) == str(HORIZONTE_PLOT))
            ].copy()
            if df_st.empty:
                continue
            plot_fdc_inset(ax_in, df_st, title=st)

        elif plot_kind == "seasonal_box_gcms":
            if df_season is None:
                continue
            plot_sazonal_box_modelos_inset(
                ax_in=ax_in,
                df_season=df_season,
                estacao=st,
                horizonte_sim=HORIZONTE_SIM,
                horizonte_obs=HORIZONTE_OBS,
                show_obs=SHOW_OBS,
                show_ens=SHOW_ENSEMBLE,
                ylog=SEASON_YLOG
            )

        else:  # seasonal_violin_interannual
            if df_box is None:
                continue
            sim_fonte = violin_sim_fonte if violin_sim_fonte is not None else "ENSEMBLE"
            sim_modelo = violin_sim_modelo if violin_sim_modelo is not None else "ENSEMBLE_MEAN"

            plot_sazonal_violin_interanual_inset(
                ax_in=ax_in,
                df_box=df_box,
                estacao=st,
                horizonte_sim=HORIZONTE_SIM,
                horizonte_obs=HORIZONTE_OBS,
                sim_fonte=sim_fonte,
                sim_modelo=sim_modelo,
                ylog=SEASON_YLOG
            )

        x_st, y_st = est_fig[st]
        x0, y0 = nearest_corner_of_inset(center, INSET_W, INSET_H, (x_st, y_st))
        fig.lines.append(plt.Line2D(
            [x0, x_st], [y0, y_st],
            transform=fig.transFigure,
            color=CALL_OUT_COLOR,
            linewidth=CALL_OUT_LW,
            alpha=CALL_OUT_ALPHA
        ))

    fig.savefig(out_png, dpi=DPI, bbox_inches="tight")
    plt.close(fig)
    print(f"✅ Figura salva: {out_png}")


# =============================================================================
# MAIN
# =============================================================================
def main() -> None:
    meta = json.loads(BASE_META.read_text(encoding="utf-8"))
    code_field = meta["station_code_aux_field"]  # "_cod_str"

    # Ler camadas consolidadas
    gdf_minis = gpd.read_file(BASE_GPKG, layer=meta["layers"]["minis"])
    gdf_sub   = gpd.read_file(BASE_GPKG, layer=meta["layers"]["subbacias"])
    gdf_hidro = gpd.read_file(BASE_GPKG, layer=meta["layers"]["hidrografia"])
    gdf_cid   = gpd.read_file(BASE_GPKG, layer=meta["layers"]["cidades"])
    gdf_est   = gpd.read_file(BASE_GPKG, layer=meta["layers"]["estacoes"])

    # -------------------------------------------------------------------------
    # Estações em coords de figura (ax temporário)
    # -------------------------------------------------------------------------
    fig_tmp = plt.figure(figsize=FIGSIZE, dpi=DPI)
    ax_tmp = fig_tmp.add_axes(AX_RECT)
    gdf_minis.boundary.plot(ax=ax_tmp, color=COLOR_MINIS, linewidth=LW_MINIS, alpha=0.55, zorder=1)
    if "SB_LBL" in gdf_sub.columns:
        gdf_sub.plot(ax=ax_tmp, column="SB_LBL", alpha=ALPHA_SUB_FILL,
                     linewidth=LW_SUB_EDGE, edgecolor=COLOR_SUB_EDGE,
                     zorder=2, legend=False)
    else:
        gdf_sub.boundary.plot(ax=ax_tmp, color=COLOR_SUB_EDGE, linewidth=LW_SUB_EDGE, zorder=2)
    plot_hidrografia(ax_tmp, gdf_hidro)
    plot_cidades(ax_tmp, gdf_cid, nome_field="CID_NM")
    plot_estacoes(ax_tmp, gdf_est, code_field=code_field)
    ax_tmp.set_axis_off()

    est_fig: Dict[str, Tuple[float, float]] = {}
    for _, r in gdf_est.iterrows():
        cod = str(r.get(code_field, "")).strip()
        if cod:
            est_fig[cod] = data_to_figcoords(ax_tmp, r.geometry.x, r.geometry.y)

    plt.close(fig_tmp)

    # -------------------------------------------------------------------------
    # Ler dados conforme modo
    # -------------------------------------------------------------------------
    plot_kind = PLOT_KIND.lower().strip()
    print(f"[INFO] PLOT_KIND = {plot_kind}")
    print(f"[INFO] GENERATE_APPENDIX_PER_MODEL = {GENERATE_APPENDIX_PER_MODEL}")
    print(f"[INFO] OUT_APPENDIX_DIR = {OUT_APPENDIX_DIR}")

    if plot_kind == "fdc":
        df_fdc = pd.read_csv(CSV_FDC, sep=FDC_SEP, dtype=str)

        # normaliza
        for c in ["estacao_obs", "modelo", "horizonte"]:
            if c in df_fdc.columns:
                df_fdc[c] = df_fdc[c].astype(str).str.strip()

        # numéricos
        df_fdc[EXC_COL] = pd.to_numeric(df_fdc[EXC_COL], errors="coerce")
        df_fdc[QSIM_COL] = pd.to_numeric(df_fdc[QSIM_COL], errors="coerce")
        if QOBS_COL in df_fdc.columns:
            df_fdc[QOBS_COL] = pd.to_numeric(df_fdc[QOBS_COL], errors="coerce")

        # DIAG
        print("[DIAG][FDC] colunas:", list(df_fdc.columns))
        print("[DIAG][FDC] horizontes únicos (amostra):", sorted(df_fdc["horizonte"].dropna().unique())[:30], "...")
        print("[DIAG][FDC] modelos únicos (amostra):", sorted(df_fdc["modelo"].dropna().unique())[:30], "...")

        # 1) mapa principal (ensemble)
        out_main = OUT_DIR / f"MAPA_FDC_{MODELO_PLOT}_{HORIZONTE_PLOT}_LAYOUT_MANUAL.png"
        render_map(
            gdf_minis=gdf_minis, gdf_sub=gdf_sub, gdf_hidro=gdf_hidro, gdf_cid=gdf_cid, gdf_est=gdf_est,
            code_field=code_field, est_fig=est_fig,
            plot_kind=plot_kind, out_png=out_main,
            df_fdc=df_fdc, fdc_modelo=MODELO_PLOT
        )

        # 2) apêndice: 1 mapa por GCM
        if GENERATE_APPENDIX_PER_MODEL:
            df_h = df_fdc[df_fdc["horizonte"] == HORIZONTE_PLOT].copy()
            if df_h.empty:
                print(f"[WARN][FDC] Nenhuma linha com horizonte='{HORIZONTE_PLOT}'. Usando arquivo inteiro (fallback).")
                df_h = df_fdc

            # se existir coluna 'fonte', tenta filtrar fonte==GCM; senão faz por exclusão
            if "fonte" in df_h.columns:
                df_h["fonte"] = df_h["fonte"].astype(str).str.strip()
                fontes = sorted(df_h["fonte"].dropna().unique())
                print("[DIAG][FDC] fontes únicas:", fontes)

                df_gcm = df_h[df_h["fonte"].str.upper() == "GCM"].copy()
                if df_gcm.empty:
                    print("[WARN][FDC] Não encontrei fonte=='GCM' no CSV FDC. Vou usar fallback por exclusão (OBS/ENSEMBLE).")
                    df_gcm = df_h
            else:
                df_gcm = df_h

            mlist = sorted(set(df_gcm["modelo"].dropna().unique()))
            # remove ensemble/obs e o modelo principal
            mlist = [m for m in mlist if (m != MODELO_PLOT) and (not _looks_like_ensemble_or_obs(m))]

            print(f"[INFO][FDC] Modelos GCM para apêndice = {len(mlist)}")
            if not mlist:
                print("[WARN][FDC] Lista de GCMs vazia. Verifique o DIAG acima para nomes reais.")
            else:
                for m in mlist:
                    m_safe = _sanitize_name(m)
                    out_mod = OUT_APPENDIX_DIR / f"MAPA_FDC_{m_safe}_{HORIZONTE_PLOT}_LAYOUT_MANUAL.png"
                    render_map(
                        gdf_minis=gdf_minis, gdf_sub=gdf_sub, gdf_hidro=gdf_hidro, gdf_cid=gdf_cid, gdf_est=gdf_est,
                        code_field=code_field, est_fig=est_fig,
                        plot_kind=plot_kind, out_png=out_mod,
                        df_fdc=df_fdc, fdc_modelo=m
                    )

    elif plot_kind == "seasonal_box_gcms":
        df_season = pd.read_csv(CSV_SEASON, sep=SEASON_SEP, dtype=str)

        df_season[EST_COL] = df_season[EST_COL].astype(str).str.strip()
        df_season[HORIZ_COL] = df_season[HORIZ_COL].astype(str).str.strip()
        df_season[FONTE_COL] = df_season[FONTE_COL].astype(str).str.strip()
        df_season[MODELO_COL] = df_season[MODELO_COL].astype(str).str.strip()

        df_season[MES_COL] = pd.to_numeric(df_season[MES_COL], errors="coerce")
        df_season[Q_COL] = pd.to_numeric(df_season[Q_COL], errors="coerce")

        # DIAG (ajuda a não ter “pasta vazia”)
        print("[DIAG][SEASON-BOX] horizontes únicos (amostra):", sorted(df_season[HORIZ_COL].dropna().unique())[:30], "...")
        print("[DIAG][SEASON-BOX] fontes únicas:", sorted(df_season[FONTE_COL].dropna().unique()))

        out_main = OUT_DIR / f"MAPA_SAZ_BOX_{HORIZONTE_SIM}_LAYOUT_MANUAL.png"
        render_map(
            gdf_minis=gdf_minis, gdf_sub=gdf_sub, gdf_hidro=gdf_hidro, gdf_cid=gdf_cid, gdf_est=gdf_est,
            code_field=code_field, est_fig=est_fig,
            plot_kind=plot_kind, out_png=out_main,
            df_season=df_season
        )

        if GENERATE_APPENDIX_PER_MODEL:
            print("[INFO] seasonal_box_gcms: não gera mapas por modelo (box = distribuição intermodelo).")

    elif plot_kind == "seasonal_violin_interannual":
        df_box = pd.read_csv(CSV_SEASON_BOX, sep=SEASON_SEP, dtype=str)

        df_box[EST_COL] = df_box[EST_COL].astype(str).str.strip()
        df_box[HORIZ_COL] = df_box[HORIZ_COL].astype(str).str.strip()
        df_box[FONTE_COL] = df_box[FONTE_COL].astype(str).str.strip()
        df_box[MODELO_COL] = df_box[MODELO_COL].astype(str).str.strip()

        df_box[ANO_COL] = pd.to_numeric(df_box[ANO_COL], errors="coerce")
        df_box[MES_COL] = pd.to_numeric(df_box[MES_COL], errors="coerce")
        df_box[Q_COL_BOX] = pd.to_numeric(df_box[Q_COL_BOX], errors="coerce")

        print("[DIAG][VIOLIN] horizontes únicos (amostra):", sorted(df_box[HORIZ_COL].dropna().unique())[:30], "...")
        print("[DIAG][VIOLIN] fontes únicas:", sorted(df_box[FONTE_COL].dropna().unique()))

        # mapa principal (ENSEMBLE)
        out_main = OUT_DIR / f"MAPA_SAZ_VIOLIN_INTERANUAL_ENSEMBLE_{HORIZONTE_SIM}.png"
        render_map(
            gdf_minis=gdf_minis, gdf_sub=gdf_sub, gdf_hidro=gdf_hidro, gdf_cid=gdf_cid, gdf_est=gdf_est,
            code_field=code_field, est_fig=est_fig,
            plot_kind=plot_kind, out_png=out_main,
            df_box=df_box,
            violin_sim_fonte="ENSEMBLE",
            violin_sim_modelo="ENSEMBLE_MEAN"
        )

        # mapas individuais por modelo (GCM)
        if GENERATE_APPENDIX_PER_MODEL:
            modelos_appendix = sorted(set(
                df_box.loc[
                    (df_box[FONTE_COL] == "GCM") &
                    (df_box[HORIZ_COL] == HORIZONTE_SIM),
                    MODELO_COL
                ].dropna().unique()
            ))

            print(f"[INFO][VIOLIN] Modelos GCM para apêndice = {len(modelos_appendix)}")
            for m in modelos_appendix:
                m_safe = _sanitize_name(m)
                out_mod = OUT_APPENDIX_DIR / f"MAPA_SAZ_VIOLIN_INTERANUAL_{m_safe}_{HORIZONTE_SIM}.png"
                render_map(
                    gdf_minis=gdf_minis, gdf_sub=gdf_sub, gdf_hidro=gdf_hidro, gdf_cid=gdf_cid, gdf_est=gdf_est,
                    code_field=code_field, est_fig=est_fig,
                    plot_kind=plot_kind, out_png=out_mod,
                    df_box=df_box,
                    violin_sim_fonte="GCM",
                    violin_sim_modelo=m
                )

    else:
        raise ValueError("PLOT_KIND deve ser: 'fdc', 'seasonal_box_gcms' ou 'seasonal_violin_interannual'.")


if __name__ == "__main__":
    main()

In [ ]:
# -*- coding: utf-8 -*-
"""
===============================================================================
SCRIPT 02 — MAPA FINAL + INSETS (FDC e SAZONALIDADE) — LAYOUT FIXO (manual)
VERSÃO CORRIGIDA

OBJETIVO (1 rodada):
- Gerar TODOS os mapas possíveis:
  (A) FDC: todos os horizontes, para ENSEMBLE e para TODOS os GCMs
  (B) SAZONALIDADE BOX (GCMs): todos os horizontes (sem "por modelo", pois o box é intermodelo)
  (C) SAZONALIDADE VIOLIN/BOX interanual: todos os horizontes, para ENSEMBLE e para TODOS os GCMs

SAÍDAS (organizadas em pastas):
OUT_DIR/
  FDC/<HORIZONTE>/
    ensemble/MAPA_FDC_ENSEMBLE_MEAN_<HORIZ>.png
    modelos/MAPA_FDC_<GCM>_<HORIZ>.png

  SAZONAL_BOX/<HORIZONTE>/
    MAPA_SAZ_BOX_<HORIZ>.png

  SAZONAL_VIOLIN_INTERANUAL/<HORIZONTE>/
    ensemble/MAPA_SAZ_VIOLIN_ENS_<HORIZ>.png
    modelos/MAPA_SAZ_VIOLIN_<GCM>_<HORIZ>.png

CORREÇÕES APLICADAS:
- Verificação e instalação automática de dependências (scipy)
- Compatibilidade com diferentes versões do matplotlib
- Encoding UTF-8 explícito em todos os CSVs
- Melhor tratamento de erros e validações
- Otimização de memória aprimorada
- Mensagens de progresso mais detalhadas

DPI=300 mantido.
===============================================================================
"""

from __future__ import annotations

import gc
import json
import sys
import warnings
from pathlib import Path
from typing import Dict, Tuple, Optional, List

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl

# Suprimir warnings desnecessários
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)


# =============================================================================
# VERIFICAÇÃO E INSTALAÇÃO DE DEPENDÊNCIAS
# =============================================================================
def verificar_scipy():
    """Verifica se scipy está instalado, caso contrário tenta instalar"""
    try:
        import scipy
        return True
    except ImportError:
        print("[AVISO] scipy não encontrado. Tentando instalar...")
        try:
            import subprocess
            subprocess.check_call([sys.executable, "-m", "pip", "install", "scipy"])
            print("[OK] scipy instalado com sucesso!")
            return True
        except Exception as e:
            print(f"[ERRO] Não foi possível instalar scipy: {e}")
            print("[INFO] Interpolação suave será desabilitada.")
            return False

SCIPY_DISPONIVEL = verificar_scipy()


# =============================================================================
# ENTRADAS: BASE (saída do Script 01)
# =============================================================================
BASE_DIR = Path(r"E:\Base_Cartografica_GPKG\_base")
BASE_GPKG = BASE_DIR / "final_base.gpkg"
BASE_META = BASE_DIR / "base_meta.json"

# Saída (raiz)
OUT_DIR = Path(r"E:\RESULTADOS_AB2\SSP5-85\Figuras_FDC_2")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# =============================================================================
# LAYOUT FIGURA
# =============================================================================
# Convertendo 23cm x 14.5cm para polegadas (1 inch = 2.54 cm)
FIGSIZE = (23 / 2.54, 15 / 2.54)  # 23cm x 14.5cm = ~9.06" x 5.71"
DPI = 300
AX_RECT = [0.05, 0.08, 0.90, 0.84]

INSET_W = 0.21
INSET_H = 0.18

# =============================================================================
# FDC
# =============================================================================
CSV_FDC = Path(r"E:\RESULTADOS_AB2\SSP5-85\FDC_Projecoes_ssp585\FDC_consolidado_obs_ref_e_futuro.csv")
FDC_SEP = ";"
EXC_COL = "exc"
QSIM_COL = "q_sim"
QOBS_COL = "q_obs_ref"
FDC_YLOG = True

# =============================================================================
# SAZONALIDADE — BOX (GCMs) — CSV CONSOLIDADO (formato longo)
# =============================================================================
SEASON_DIR = Path(r"E:\RESULTADOS_AB2\SSP5-85\SAZONALIDADE")
CSV_SEASON = SEASON_DIR / "SAZONALIDADE_mensal_consolidada.csv"
SEASON_SEP = ";"

EST_COL    = "estacao_obs"
HORIZ_COL  = "horizonte"
FONTE_COL  = "fonte"
MODELO_COL = "modelo"
MES_COL    = "mes"
Q_COL      = "Q_medio"

HORIZONTE_OBS = "REF_1980_2023"
SHOW_OBS = True
SHOW_ENSEMBLE = True
SEASON_YLOG = False

# Qual ensemble usar na climatologia (linha) do inset BOX
SEASON_ENSEMBLE_MODEL_CLIM = "ENSEMBLE_CLIM_POS_MEAN"      # ou "ENSEMBLE_CLIM_POS_MEDIAN"

# =============================================================================
# SAZONALIDADE — VIOLIN/BOX INTERANUAL (OBS + ENSEMBLE + GCM)
# =============================================================================
CSV_SEASON_BOX = SEASON_DIR / "SAZONALIDADE_boxplot_interanual_obs_ensemble.csv"

ANO_COL = "ano"
Q_COL_BOX = "Q_mensal"

# Qual ensemble usar no interanual (ano-mês)
SEASON_ENSEMBLE_MODEL_INTERANUAL = "ENSEMBLE_MENSAL_POS_MEAN"  # ou "ENSEMBLE_MENSAL_POS_MEDIAN"

# Compatibilidade com diferentes versões do matplotlib
try:
    MONTH_CMAP = mpl.colormaps["tab20"]
except (AttributeError, KeyError):
    try:
        MONTH_CMAP = plt.cm.get_cmap("tab20")
    except:
        MONTH_CMAP = plt.cm.tab20

MONTH_LABELS = ["Jan","Fev","Mar","Abr","Mai","Jun","Jul","Ago","Set","Out","Nov","Dez"]

# =============================================================================
# POSIÇÕES FIXAS (slots) — coordenadas da FIGURA (0–1)
# =============================================================================
INSET_POS: Dict[str, Tuple[float, float]] = {
    "65208000": (0.15, 0.86),
    "65060000": (0.52, 0.88),
    "65035000": (0.82, 0.89),
    "65310000": (0.08, 0.55),
    "65155000": (0.82, 0.55),
    "65220000": (0.32, 0.12),
    "65295000": (0.06, 0.18),
    "65175000": (0.58, 0.15),
    "65095000": (0.82, 0.28),
}

# =============================================================================
# CARTOGRAFIA (mantém padrão)
# =============================================================================
COLOR_MINIS = "#8a8a8a"
COLOR_SUB_EDGE = "#cc0000"

COLOR_H_13 = "#1f4e79"
COLOR_H_4  = "#4f81bd"
COLOR_H_5  = "#9dc3e6"

LW_MINIS = 0.25
LW_SUB_EDGE = 1.10
LW_H_13 = 0.75
LW_H_4  = 0.95
LW_H_5  = 1.10

ALPHA_SUB_FILL = 0.22

CALL_OUT_COLOR = "crimson"
CALL_OUT_LW = 0.9
CALL_OUT_ALPHA = 0.85


# =============================================================================
# HELPERS
# =============================================================================
def garantir_pasta(p: Path) -> None:
    """Cria pasta se não existir"""
    p.mkdir(parents=True, exist_ok=True)

def _sanitize_name(s: str) -> str:
    """Remove caracteres inválidos de nomes de arquivo"""
    s = str(s).strip()
    for ch in ["/", "\\", ":", "*", "?", '"', "<", ">", "|"]:
        s = s.replace(ch, "_")
    s = s.replace(" ", "_")
    return s

def _looks_like_ensemble_or_obs(model_name: str) -> bool:
    """Verifica se o nome do modelo parece ser ensemble ou observado"""
    m = str(model_name).strip().upper()
    if m in {"OBS", "OBS_REF", "OBSERVADO", "OBSERVED"}:
        return True
    if "ENSEMBLE" in m:
        return True
    return False

def data_to_figcoords(ax, x, y) -> Tuple[float, float]:
    """Converte coordenadas de dados para coordenadas da figura"""
    xy_disp = ax.transData.transform((x, y))
    xy_fig = ax.figure.transFigure.inverted().transform(xy_disp)
    return float(xy_fig[0]), float(xy_fig[1])

def make_inset(fig, center_xy, w, h):
    """Cria um inset na figura"""
    cx, cy = center_xy
    left = cx - w / 2
    bottom = cy - h / 2
    return fig.add_axes([left, bottom, w, h])

def inset_corners(center_xy, w, h):
    """Retorna os cantos de um inset"""
    cx, cy = center_xy
    left = cx - w/2
    right = cx + w/2
    bottom = cy - h/2
    top = cy + h/2
    return [(left, bottom), (left, top), (right, bottom), (right, top)]

def nearest_corner_of_inset(center_xy, w, h, target_xy):
    """Encontra o canto mais próximo do inset em relação a um ponto"""
    tx, ty = target_xy
    corners = inset_corners(center_xy, w, h)
    return min(corners, key=lambda p: (p[0]-tx)**2 + (p[1]-ty)**2)

def detectar_campo_ordem(gdf_h: gpd.GeoDataFrame) -> Optional[str]:
    """Detecta o campo de ordem de Strahler na hidrografia"""
    cols = list(gdf_h.columns)
    if "_ord2" in cols:
        return "_ord2"
    if "_ord" in cols:
        return "_ord"
    lower_map = {c.lower(): c for c in cols}
    candidatos = ["nuordemcda", "nuordemdca", "nuordem", "ordem", "order", "ord", "strahler", "str_order"]
    for c in candidatos:
        if c.lower() in lower_map:
            return lower_map[c.lower()]
    heur = [c for c in cols if ("ord" in c.lower()) or ("strah" in c.lower())]
    return heur[0] if heur else None


# =============================================================================
# OTIMIZAÇÃO: hidrografia pré-processada 1x (evita assign/copy por figura)
# =============================================================================
def preparar_hidrografia(gdf_h: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """
    Retorna hidrografia pronta para plot:
    - com coluna _ord2 (int) e filtrada 1..5, se possível
    - caso não haja campo de ordem, retorna o original (sem alteração)
    """
    if gdf_h.empty:
        return gdf_h

    campo = detectar_campo_ordem(gdf_h)
    if campo is None:
        return gdf_h

    if campo == "_ord2":
        g = gdf_h
    else:
        ords = pd.to_numeric(gdf_h[campo], errors="coerce").fillna(-1).astype(int)
        g = gdf_h.copy()
        g["_ord2"] = ords.values

    g = g[(g["_ord2"] >= 1) & (g["_ord2"] <= 5)].copy()
    return g


def plot_hidrografia(ax, gdf_h_prepared: gpd.GeoDataFrame):
    """
    Não cria cópias internas: espera gdf_h já preparado.
    """
    if gdf_h_prepared.empty:
        return

    if "_ord2" not in gdf_h_prepared.columns:
        gdf_h_prepared.plot(ax=ax, color=COLOR_H_4, linewidth=0.8, alpha=0.9, zorder=3)
        return

    g13 = gdf_h_prepared[gdf_h_prepared["_ord2"].isin([1, 2, 3])]
    g4  = gdf_h_prepared[gdf_h_prepared["_ord2"] == 4]
    g5  = gdf_h_prepared[gdf_h_prepared["_ord2"] == 5]

    if len(g13):
        g13.plot(ax=ax, color=COLOR_H_13, linewidth=LW_H_13, alpha=0.9, zorder=3)
    if len(g4):
        g4.plot(ax=ax, color=COLOR_H_4, linewidth=LW_H_4, alpha=0.7, zorder=3)
    if len(g5):
        g5.plot(ax=ax, color=COLOR_H_5, linewidth=LW_H_5, alpha=0.5, zorder=3)


def plot_estacoes(ax, gdf_est: gpd.GeoDataFrame, code_field: str):
    """Plota estações no mapa"""
    gdf_est.plot(ax=ax, marker="v", markersize=70, color="black",
                 edgecolor="white", linewidth=0.6, zorder=6)
    
    # Ajustes específicos para estações sobrepostas
    ajustes_estacoes = {
        "65208000": (-20, 10),   # Desloca esquerda e cima
        "65060000": (6, -8),   # Desloca direita e baixo
        "65035000": (5, -8),   # Desloca baixo
    }
    
    for _, r in gdf_est.iterrows():
        txt = str(r.get(code_field, "")).strip()
        if not txt:
            continue
        
        # Verifica se há ajuste específico para esta estação
        if txt in ajustes_estacoes:
            offset = ajustes_estacoes[txt]
            ax.annotate(
                txt,
                xy=(r.geometry.x, r.geometry.y),
                xytext=offset,
                textcoords="offset points",
                fontsize=8,
                color="black",
                zorder=7,
                bbox=dict(boxstyle="round,pad=0.15", facecolor="white", 
                         edgecolor="gray", linewidth=0.3, alpha=0.85)
            )
        else:
            # Posição padrão
            ax.annotate(
                txt,
                xy=(r.geometry.x, r.geometry.y),
                xytext=(4, 4),
                textcoords="offset points",
                fontsize=8,
                color="black",
                zorder=7,
                bbox=dict(boxstyle="round,pad=0.15", facecolor="white", 
                         edgecolor="gray", linewidth=0.3, alpha=0.85)
            )

def plot_cidades(ax, gdf_cid: gpd.GeoDataFrame, nome_field: str):
    """Plota cidades no mapa"""
    if gdf_cid.empty or nome_field not in gdf_cid.columns:
        return
    gdf_cid.plot(ax=ax, marker="o", markersize=35, color="white",
                 edgecolor="black", linewidth=0.8, zorder=7)
    for _, r in gdf_cid.iterrows():
        ax.annotate(
            str(r[nome_field]),
            xy=(r.geometry.x, r.geometry.y),
            xytext=(5, -8),
            textcoords="offset points",
            fontsize=8,
            color="black",
            zorder=8,
        )


# =============================================================================
# INSETS: FDC
# =============================================================================
def plot_fdc_inset(ax_in, df_fdc: pd.DataFrame, title: str):
    """Plota curva de permanência (FDC) no inset"""
    df_fdc = df_fdc.sort_values(EXC_COL).copy()

    def pos(s):
        s = pd.to_numeric(s, errors="coerce")
        return s.where(s > 0)

    x = pd.to_numeric(df_fdc[EXC_COL], errors="coerce")
    y_sim = pos(df_fdc[QSIM_COL])

    if QOBS_COL in df_fdc.columns:
        y_obs = pos(df_fdc[QOBS_COL])
        if y_obs.notna().any():
            ax_in.plot(x, y_obs, lw=1.2, color="black", label="Obs")

    ax_in.plot(x, y_sim, lw=1.1, label="Sim")

    ax_in.set_xlim(0, 100)
    if FDC_YLOG:
        ax_in.set_yscale("log")

    ax_in.set_title(title, fontsize=7, pad=2)
    ax_in.set_xlabel("Excedência (%)", fontsize=6)
    ax_in.set_ylabel("Q (m³/s)", fontsize=6)
    ax_in.tick_params(labelsize=6, length=2)
    ax_in.grid(True, alpha=0.22, linewidth=0.5)
    ax_in.legend(fontsize=5.4, loc="best", frameon=True)


# =============================================================================
# OTIMIZAÇÃO: precompute lookups para insets (evita filtros repetidos)
# =============================================================================
def build_fdc_lookup(df_fdc: pd.DataFrame) -> Dict[Tuple[str, str, str], pd.DataFrame]:
    """
    key = (horizonte, modelo, estacao_obs) -> df_st (com colunas necessárias)
    """
    lookup: Dict[Tuple[str, str, str], pd.DataFrame] = {}
    grp = df_fdc.groupby(["horizonte", "modelo", "estacao_obs"], sort=False)
    for (h, m, st), g in grp:
        gg = g[[EXC_COL, QSIM_COL, QOBS_COL]].copy() if QOBS_COL in g.columns else g[[EXC_COL, QSIM_COL]].copy()
        lookup[(str(h), str(m), str(st))] = gg
    return lookup

def build_violin_lookup(df_box: pd.DataFrame) -> Dict[Tuple[str, str, str, str], pd.DataFrame]:
    """
    key = (horizonte, fonte, modelo, estacao_obs) -> df_est (mes/Q_mensal)
    """
    lookup: Dict[Tuple[str, str, str, str], pd.DataFrame] = {}
    grp = df_box.groupby([HORIZ_COL, FONTE_COL, MODELO_COL, EST_COL], sort=False)
    for (h, fonte, m, st), g in grp:
        gg = g[[MES_COL, Q_COL_BOX]].copy()
        lookup[(str(h), str(fonte), str(m), str(st))] = gg
    return lookup

def build_season_box_lookup(df_season: pd.DataFrame) -> Dict[Tuple[str, str], pd.DataFrame]:
    """
    key = (horizonte, estacao_obs) -> df_est (fonte/modelo/mes/Q_medio)
    """
    lookup: Dict[Tuple[str, str], pd.DataFrame] = {}
    grp = df_season.groupby([HORIZ_COL, EST_COL], sort=False)
    for (h, st), g in grp:
        gg = g[[FONTE_COL, MODELO_COL, MES_COL, Q_COL]].copy()
        lookup[(str(h), str(st))] = gg
    return lookup


# =============================================================================
# INSETS: SAZONALIDADE — BOX (GCMs) + Obs + Ensemble
# =============================================================================
def plot_sazonal_box_modelos_inset_from_group(
    ax_in: plt.Axes,
    g_est: pd.DataFrame,
    horizonte_sim: str,
    horizonte_obs: str = "REF_1980_2023",
    show_obs: bool = True,
    show_ens: bool = True,
    ylog: bool = False,
    ensemble_model: str = "ENSEMBLE_CLIM_POS_MEAN",
) -> None:
    """Plota sazonalidade em boxplot (GCMs) com obs e ensemble"""
    if g_est is None or g_est.empty:
        return

    # g_est vem por (horiz, estacao) do lookup; não deve ter HORIZ_COL,
    # mas mantemos o fallback se existir.
    if HORIZ_COL in g_est.columns:
        gcm = g_est[(g_est[FONTE_COL] == "GCM") & (g_est[HORIZ_COL] == horizonte_sim)]
    else:
        gcm = g_est[g_est[FONTE_COL] == "GCM"]

    boxes = []
    for m in range(1, 13):
        vals = gcm.loc[gcm[MES_COL] == m, Q_COL].dropna().values.astype(float)
        boxes.append(vals)

    ax_in.boxplot(
        boxes,
        positions=np.arange(1, 13),
        widths=0.55,
        showfliers=False
    )

    if show_obs:
        obs = g_est[(g_est[FONTE_COL] == "OBS") & (g_est[MODELO_COL] == "OBS")]
        if not obs.empty:
            obs = obs.dropna(subset=[MES_COL, Q_COL]).sort_values(MES_COL)
            ax_in.plot(obs[MES_COL], obs[Q_COL], color="black", lw=1.2, label="Obs")

    if show_ens:
        ens = g_est[(g_est[FONTE_COL] == "ENSEMBLE") & (g_est[MODELO_COL] == ensemble_model)]
        if not ens.empty:
            ens = ens.dropna(subset=[MES_COL, Q_COL]).sort_values(MES_COL)
            ax_in.plot(ens[MES_COL], ens[Q_COL], lw=1.2, label="Ensemble")

    ax_in.set_xlim(0.5, 12.5)
    ax_in.set_xticks([1, 3, 5, 7, 9, 11])
    ax_in.set_xlabel("Mês", fontsize=6)
    ax_in.set_ylabel("Q (m³/s)", fontsize=6)
    ax_in.tick_params(labelsize=6, length=2)
    ax_in.grid(True, alpha=0.22, linewidth=0.5)

    if ylog:
        ymin = np.inf
        ymax = 0.0
        for arr in boxes:
            if arr.size:
                arrp = arr[arr > 0]
                if arrp.size:
                    ymin = min(ymin, float(arrp.min()))
                    ymax = max(ymax, float(arrp.max()))
        if np.isfinite(ymin) and ymax > 0:
            ax_in.set_yscale("log")
            ax_in.set_ylim(max(ymin, 0.01), ymax * 1.2)

    handles, labels = ax_in.get_legend_handles_labels()
    if labels:
        ax_in.legend(fontsize=5.2, loc="best", frameon=True)


# =============================================================================
# INSETS: SAZONALIDADE — VIOLIN/BOX INTERANUAL (linhas = MÉDIA)
# =============================================================================
def plot_sazonal_violin_interanual_inset_from_lookup(
    ax_in: plt.Axes,
    violin_lookup: Dict[Tuple[str, str, str, str], pd.DataFrame],
    estacao: str,
    horizonte_sim: str,
    horizonte_obs: str,
    sim_fonte: str,
    sim_modelo: str,
    ylog: bool = False
) -> None:
    """Plota sazonalidade interanual com violin/boxplot"""
    d_sim = violin_lookup.get((str(horizonte_sim), str(sim_fonte), str(sim_modelo), str(estacao)))
    if d_sim is None or d_sim.empty:
        return

    boxes = []
    sim_mean = []
    for m in range(1, 13):
        vals = d_sim.loc[d_sim[MES_COL] == m, Q_COL_BOX].dropna().values.astype(float)
        boxes.append(vals)
        sim_mean.append(np.nanmean(vals) if vals.size else np.nan)
    sim_mean = np.array(sim_mean, dtype=float)

    x = np.arange(1, 13)

    vparts = ax_in.violinplot(
        boxes,
        positions=x,
        widths=0.85,
        showmeans=False,
        showmedians=False,
        showextrema=False
    )
    for i, body in enumerate(vparts["bodies"]):
        color = MONTH_CMAP(i)
        body.set_facecolor(color)
        body.set_edgecolor("black")
        body.set_alpha(0.55)
        body.set_linewidth(0.6)

    ax_in.boxplot(
        boxes,
        positions=x,
        widths=0.22,
        showfliers=False,
        patch_artist=True,
        boxprops=dict(facecolor="white", edgecolor="black", linewidth=0.9),
        medianprops=dict(color="black", linewidth=1.0),
        whiskerprops=dict(color="black", linewidth=0.9),
        capprops=dict(color="black", linewidth=0.9),
    )

    d_obs = violin_lookup.get((str(horizonte_obs), "OBS", "OBS", str(estacao)))
    obs_mean = np.full(12, np.nan, dtype=float)
    if d_obs is not None and not d_obs.empty:
        for m in range(1, 13):
            vals = d_obs.loc[d_obs[MES_COL] == m, Q_COL_BOX].dropna().values.astype(float)
            obs_mean[m-1] = np.nanmean(vals) if vals.size else np.nan

    def _plot_smooth_line(xv, yv, label, color=None, linestyle="-"):
        """Plota linha com interpolação suave se scipy disponível"""
        mask = np.isfinite(yv)
        if mask.sum() < 3:
            ax_in.plot(xv[mask], yv[mask], label=label, color=color, lw=1.2, linestyle=linestyle)
            return
        
        if not SCIPY_DISPONIVEL:
            # Fallback: linha simples sem interpolação
            ax_in.plot(xv[mask], yv[mask], label=label, color=color, lw=1.2, linestyle=linestyle)
            return
            
        try:
            from scipy.interpolate import PchipInterpolator
            xs = np.linspace(1, 12, 240)
            f = PchipInterpolator(xv[mask], yv[mask])
            ys = f(xs)
            ax_in.plot(xs, ys, label=label, color=color, lw=1.2, linestyle=linestyle)
        except Exception as e:
            # Fallback em caso de erro na interpolação
            ax_in.plot(xv[mask], yv[mask], label=label, color=color, lw=1.2, linestyle=linestyle)

    # Nome simplificado para ensemble
    if "ENSEMBLE" in str(sim_modelo).upper():
        label_sim = "Ensemble (média)"
    else:
        label_sim = f"{sim_modelo} (média)"
    
    _plot_smooth_line(x, sim_mean, label=label_sim, color=None, linestyle="-")
    _plot_smooth_line(x, obs_mean, label="Obs (média)", color="black", linestyle="-")

    ax_in.set_xlim(0.5, 12.5)
    ax_in.set_xticks(np.arange(1, 13))
    ax_in.set_xticklabels(MONTH_LABELS, fontsize=5.2, rotation=35, ha="right")
    ax_in.set_xlabel("")
    ax_in.set_ylabel("Q (m³/s)", fontsize=6)
    ax_in.set_title(str(estacao), fontsize=7, pad=2)
    ax_in.tick_params(axis="y", labelsize=6, length=2)
    ax_in.grid(True, alpha=0.22, linewidth=0.5)

    if ylog:
        ymin = np.inf
        ymax = 0.0
        for arr in boxes:
            if arr.size:
                arrp = arr[arr > 0]
                if arrp.size:
                    ymin = min(ymin, float(arrp.min()))
                    ymax = max(ymax, float(arrp.max()))
        for v in np.r_[sim_mean, obs_mean]:
            if np.isfinite(v) and v > 0:
                ymin = min(ymin, float(v))
                ymax = max(ymax, float(v))

        if np.isfinite(ymin) and ymax > 0:
            ax_in.set_yscale("log")
            ax_in.set_ylim(max(ymin, 0.01), ymax * 1.2)

    handles, labels = ax_in.get_legend_handles_labels()
    if labels:
        ax_in.legend(fontsize=5.0, loc="best", frameon=True)


# =============================================================================
# FUNÇÃO: desenha e salva 1 mapa
# =============================================================================
def render_map(
    gdf_minis: gpd.GeoDataFrame,
    gdf_sub: gpd.GeoDataFrame,
    gdf_hidro_prepared: gpd.GeoDataFrame,
    gdf_cid: gpd.GeoDataFrame,
    gdf_est: gpd.GeoDataFrame,
    code_field: str,
    est_fig: Dict[str, Tuple[float, float]],
    out_png: Path,
    kind: str,                 # "fdc" | "seasonal_box_gcms" | "seasonal_violin_interannual"
    horizon: str,
    # lookups
    fdc_lookup: Optional[Dict[Tuple[str, str, str], pd.DataFrame]] = None,
    season_box_lookup: Optional[Dict[Tuple[str, str], pd.DataFrame]] = None,
    violin_lookup: Optional[Dict[Tuple[str, str, str, str], pd.DataFrame]] = None,
    # seleção
    model: Optional[str] = None,
    sim_fonte: Optional[str] = None,
    # sazonal
    ensemble_model_clim: str = "ENSEMBLE_CLIM_POS_MEAN",
) -> None:
    """Renderiza e salva um mapa completo com insets"""
    fig = plt.figure(figsize=FIGSIZE, dpi=DPI)
    ax = fig.add_axes(AX_RECT)

    gdf_minis.boundary.plot(ax=ax, color=COLOR_MINIS, linewidth=LW_MINIS, alpha=0.55, zorder=1)

    if "SB_LBL" in gdf_sub.columns:
        gdf_sub.plot(
            ax=ax,
            column="SB_LBL",
            alpha=ALPHA_SUB_FILL,
            linewidth=LW_SUB_EDGE,
            edgecolor=COLOR_SUB_EDGE,
            zorder=2,
            legend=False
        )
    else:
        gdf_sub.boundary.plot(ax=ax, color=COLOR_SUB_EDGE, linewidth=LW_SUB_EDGE, zorder=2)

    plot_hidrografia(ax, gdf_hidro_prepared)
    plot_cidades(ax, gdf_cid, nome_field="CID_NM")
    plot_estacoes(ax, gdf_est, code_field=code_field)

    if "SB_LBL" in gdf_sub.columns:
        gdf_sub_tmp = gdf_sub.copy()
        gdf_sub_tmp["centroid"] = gdf_sub_tmp.geometry.centroid
        
        # Dicionário de ajustes específicos (xytext offset)
        ajustes_pos = {
            "SB-01": (-10, -10),  # Desloca esquerda e para baixo
            "SB-09": (-18, -15),  # Desloca esquerda e para baixo
            "SB-04": (8, -19),    # Desloca para baixo
            "SB-03": (8, 15),    # Desloca para direira e baixo
            "SB-02": (0, -15),    # Desloca para baixo
            "SB-08": (0, 15),    # Desloca para cima
        }
        
        for _, r in gdf_sub_tmp.iterrows():
            lbl = str(r["SB_LBL"])
            
            # Verifica se há ajuste específico para este label
            if lbl in ajustes_pos:
                offset = ajustes_pos[lbl]
                ax.annotate(
                    lbl,
                    xy=(r["centroid"].x, r["centroid"].y),
                    xytext=offset,
                    textcoords="offset points",
                    ha="center", va="center",
                    fontsize=10, fontweight="bold",
                    color="#7a0000", zorder=10,
                    bbox=dict(boxstyle="round,pad=0.2", facecolor="white", 
                             edgecolor="#cc0000", linewidth=0.4, alpha=0.75)
                )
            else:
                # Outros labels mantêm posição central
                ax.annotate(
                    lbl,
                    xy=(r["centroid"].x, r["centroid"].y),
                    ha="center", va="center",
                    fontsize=10, fontweight="bold",
                    color="#7a0000", zorder=10,
                    bbox=dict(boxstyle="round,pad=0.2", facecolor="white", 
                             edgecolor="#cc0000", linewidth=0.4, alpha=0.75)
                )

    ax.set_axis_off()

    for st, center in INSET_POS.items():
        if st not in est_fig:
            continue

        ax_in = make_inset(fig, center, INSET_W, INSET_H)

        if kind == "fdc":
            if fdc_lookup is None or model is None:
                continue
            df_st = fdc_lookup.get((str(horizon), str(model), str(st)))
            if df_st is None or df_st.empty:
                continue
            plot_fdc_inset(ax_in, df_st, title=st)

        elif kind == "seasonal_box_gcms":
            if season_box_lookup is None:
                continue
            g_est = season_box_lookup.get((str(horizon), str(st)))
            if g_est is None or g_est.empty:
                continue
            plot_sazonal_box_modelos_inset_from_group(
                ax_in=ax_in,
                g_est=g_est,
                horizonte_sim=str(horizon),
                horizonte_obs=HORIZONTE_OBS,
                show_obs=SHOW_OBS,
                show_ens=SHOW_ENSEMBLE,
                ylog=SEASON_YLOG,
                ensemble_model=ensemble_model_clim,
            )
            ax_in.set_title(str(st), fontsize=7, pad=2)

        elif kind == "seasonal_violin_interannual":
            if violin_lookup is None or model is None or sim_fonte is None:
                continue
            plot_sazonal_violin_interanual_inset_from_lookup(
                ax_in=ax_in,
                violin_lookup=violin_lookup,
                estacao=str(st),
                horizonte_sim=str(horizon),
                horizonte_obs=HORIZONTE_OBS,
                sim_fonte=str(sim_fonte),
                sim_modelo=str(model),
                ylog=SEASON_YLOG
            )
            ax_in.set_title(str(st), fontsize=7, pad=2)
        else:
            raise ValueError(f"kind inválido: {kind}")

        x_st, y_st = est_fig[st]
        x0, y0 = nearest_corner_of_inset(center, INSET_W, INSET_H, (x_st, y_st))
        fig.lines.append(plt.Line2D(
            [x0, x_st], [y0, y_st],
            transform=fig.transFigure,
            color=CALL_OUT_COLOR,
            linewidth=CALL_OUT_LW,
            alpha=CALL_OUT_ALPHA
        ))

    fig.savefig(out_png, dpi=DPI, bbox_inches="tight")
    fig.clf()
    plt.close(fig)


# =============================================================================
# MAIN (AUTOMÁTICO)
# =============================================================================
def main() -> None:
    """Função principal: gera todos os mapas"""
    print("\n" + "="*80)
    print("SCRIPT 02 — GERAÇÃO DE MAPAS (VERSÃO CORRIGIDA)")
    print("="*80 + "\n")
    
    # Verificar arquivos de entrada
    print("[1/8] Verificando arquivos de entrada...")
    if not BASE_GPKG.exists():
        raise FileNotFoundError(f"GPKG não encontrado: {BASE_GPKG}")
    if not BASE_META.exists():
        raise FileNotFoundError(f"Metadata não encontrado: {BASE_META}")
    if not CSV_FDC.exists():
        raise FileNotFoundError(f"CSV FDC não encontrado: {CSV_FDC}")
    if not CSV_SEASON.exists():
        raise FileNotFoundError(f"CSV Sazonalidade não encontrado: {CSV_SEASON}")
    if not CSV_SEASON_BOX.exists():
        raise FileNotFoundError(f"CSV Sazonalidade BOX não encontrado: {CSV_SEASON_BOX}")
    print("    ✓ Todos os arquivos encontrados!\n")
    
    # Carregar metadata
    print("[2/8] Carregando metadata...")
    meta = json.loads(BASE_META.read_text(encoding="utf-8"))
    code_field = meta["station_code_aux_field"]
    print(f"    ✓ Campo de código: {code_field}\n")

    # Carregar camadas geoespaciais
    print("[3/8] Carregando camadas geoespaciais...")
    gdf_minis = gpd.read_file(BASE_GPKG, layer=meta["layers"]["minis"])
    print(f"    ✓ Minibacias: {len(gdf_minis)} features")
    
    gdf_sub = gpd.read_file(BASE_GPKG, layer=meta["layers"]["subbacias"])
    print(f"    ✓ Subbacias: {len(gdf_sub)} features")
    
    gdf_hidro_raw = gpd.read_file(BASE_GPKG, layer=meta["layers"]["hidrografia"])
    gdf_hidro = preparar_hidrografia(gdf_hidro_raw)
    print(f"    ✓ Hidrografia: {len(gdf_hidro)} features")
    
    gdf_cid = gpd.read_file(BASE_GPKG, layer=meta["layers"]["cidades"])
    print(f"    ✓ Cidades: {len(gdf_cid)} features")
    
    gdf_est = gpd.read_file(BASE_GPKG, layer=meta["layers"]["estacoes"])
    print(f"    ✓ Estações: {len(gdf_est)} features\n")

    # Calcular posições das estações em coordenadas de figura
    print("[4/8] Calculando posições das estações...")
    fig_tmp = plt.figure(figsize=FIGSIZE, dpi=DPI)
    ax_tmp = fig_tmp.add_axes(AX_RECT)
    gdf_minis.boundary.plot(ax=ax_tmp, color=COLOR_MINIS, linewidth=LW_MINIS, alpha=0.55, zorder=1)
    if "SB_LBL" in gdf_sub.columns:
        gdf_sub.plot(ax=ax_tmp, column="SB_LBL", alpha=ALPHA_SUB_FILL,
                     linewidth=LW_SUB_EDGE, edgecolor=COLOR_SUB_EDGE,
                     zorder=2, legend=False)
    else:
        gdf_sub.boundary.plot(ax=ax_tmp, color=COLOR_SUB_EDGE, linewidth=LW_SUB_EDGE, zorder=2)
    plot_hidrografia(ax_tmp, gdf_hidro)
    plot_cidades(ax_tmp, gdf_cid, nome_field="CID_NM")
    plot_estacoes(ax_tmp, gdf_est, code_field=code_field)
    ax_tmp.set_axis_off()

    est_fig: Dict[str, Tuple[float, float]] = {}
    for _, r in gdf_est.iterrows():
        cod = str(r.get(code_field, "")).strip()
        if cod:
            est_fig[cod] = data_to_figcoords(ax_tmp, r.geometry.x, r.geometry.y)

    fig_tmp.clf()
    plt.close(fig_tmp)
    plt.close("all")
    gc.collect()
    print(f"    ✓ {len(est_fig)} posições calculadas\n")

    # Carregar CSVs com encoding UTF-8 explícito
    print("[5/8] Carregando dados CSV...")
    df_fdc = pd.read_csv(CSV_FDC, sep=FDC_SEP, dtype=str, encoding='utf-8')
    for c in ["estacao_obs", "modelo", "horizonte"]:
        if c in df_fdc.columns:
            df_fdc[c] = df_fdc[c].astype(str).str.strip()
    df_fdc[EXC_COL] = pd.to_numeric(df_fdc[EXC_COL], errors="coerce")
    df_fdc[QSIM_COL] = pd.to_numeric(df_fdc[QSIM_COL], errors="coerce")
    if QOBS_COL in df_fdc.columns:
        df_fdc[QOBS_COL] = pd.to_numeric(df_fdc[QOBS_COL], errors="coerce")
    print(f"    ✓ FDC: {len(df_fdc)} registros")

    df_season = pd.read_csv(CSV_SEASON, sep=SEASON_SEP, dtype=str, encoding='utf-8')
    for c in [EST_COL, HORIZ_COL, FONTE_COL, MODELO_COL]:
        if c in df_season.columns:
            df_season[c] = df_season[c].astype(str).str.strip()
    df_season[MES_COL] = pd.to_numeric(df_season[MES_COL], errors="coerce")
    df_season[Q_COL] = pd.to_numeric(df_season[Q_COL], errors="coerce")
    print(f"    ✓ Sazonalidade: {len(df_season)} registros")

    df_box = pd.read_csv(CSV_SEASON_BOX, sep=SEASON_SEP, dtype=str, encoding='utf-8')
    for c in [EST_COL, HORIZ_COL, FONTE_COL, MODELO_COL]:
        if c in df_box.columns:
            df_box[c] = df_box[c].astype(str).str.strip()
    df_box[ANO_COL] = pd.to_numeric(df_box[ANO_COL], errors="coerce")
    df_box[MES_COL] = pd.to_numeric(df_box[MES_COL], errors="coerce")
    df_box[Q_COL_BOX] = pd.to_numeric(df_box[Q_COL_BOX], errors="coerce")
    print(f"    ✓ Sazonalidade BOX: {len(df_box)} registros\n")

    # Detectar e ajustar modelos ensemble
    print("[6/8] Detectando modelos ensemble...")
    ens_clim_candidates = sorted(df_season.loc[df_season[FONTE_COL] == "ENSEMBLE", MODELO_COL].dropna().unique())
    ens_int_candidates = sorted(df_box.loc[df_box[FONTE_COL] == "ENSEMBLE", MODELO_COL].dropna().unique())

    ensemble_model_clim = SEASON_ENSEMBLE_MODEL_CLIM
    if ensemble_model_clim not in ens_clim_candidates and ens_clim_candidates:
        if "ENSEMBLE_CLIM_POS_MEAN" in ens_clim_candidates:
            ensemble_model_clim = "ENSEMBLE_CLIM_POS_MEAN"
        else:
            ensemble_model_clim = ens_clim_candidates[0]
        print(f"    ⚠ Ensemble climatologia ajustado para: {ensemble_model_clim}")

    violin_model_ens = SEASON_ENSEMBLE_MODEL_INTERANUAL
    if violin_model_ens not in ens_int_candidates and ens_int_candidates:
        if "ENSEMBLE_MENSAL_POS_MEAN" in ens_int_candidates:
            violin_model_ens = "ENSEMBLE_MENSAL_POS_MEAN"
        else:
            violin_model_ens = ens_int_candidates[0]
        print(f"    ⚠ Ensemble interanual ajustado para: {violin_model_ens}")
    
    print(f"    ✓ Ensemble climatologia: {ensemble_model_clim}")
    print(f"    ✓ Ensemble interanual: {violin_model_ens}\n")

    # Construir lookups
    print("[7/8] Construindo lookups (otimização)...")
    fdc_lookup = build_fdc_lookup(df_fdc)
    print(f"    ✓ FDC lookup: {len(fdc_lookup)} combinações")
    
    violin_lookup = build_violin_lookup(df_box)
    print(f"    ✓ Violin lookup: {len(violin_lookup)} combinações")
    
    season_box_lookup = build_season_box_lookup(df_season)
    print(f"    ✓ Season box lookup: {len(season_box_lookup)} combinações")
    gc.collect()
    print()

    # Listar horizontes e modelos
    horizons_fdc = sorted([h for h in df_fdc["horizonte"].dropna().unique() if h != HORIZONTE_OBS])
    horizons_box = sorted([h for h in df_season[HORIZ_COL].dropna().unique() if h != HORIZONTE_OBS])
    horizons_violin = sorted([h for h in df_box[HORIZ_COL].dropna().unique() if h != HORIZONTE_OBS])

    fdc_models_all = sorted(df_fdc["modelo"].dropna().unique())
    fdc_models_gcm = [m for m in fdc_models_all if not _looks_like_ensemble_or_obs(m)]

    fdc_model_ens = "ENSEMBLE_MEAN"
    if fdc_model_ens not in fdc_models_all:
        ens_candidates = [m for m in fdc_models_all if "ENSEMBLE" in str(m).upper()]
        if ens_candidates:
            fdc_model_ens = ens_candidates[0]

    violin_models_gcm = sorted(df_box.loc[df_box[FONTE_COL] == "GCM", MODELO_COL].dropna().unique())

    # Criar estrutura de pastas
    out_fdc_root = OUT_DIR / "FDC"
    out_box_root = OUT_DIR / "SAZONAL_BOX"
    out_violin_root = OUT_DIR / "SAZONAL_VIOLIN_INTERANUAL"
    garantir_pasta(out_fdc_root)
    garantir_pasta(out_box_root)
    garantir_pasta(out_violin_root)

    print("[8/8] Iniciando geração de mapas...\n")
    print("="*80)
    nfig = 0

    # FDC
    print(f"\n[FDC] Gerando mapas para {len(horizons_fdc)} horizontes...")
    for i_h, h in enumerate(horizons_fdc, 1):
        print(f"  [{i_h}/{len(horizons_fdc)}] Horizonte: {h}")
        h_dir = out_fdc_root / h
        dir_ens = h_dir / "ensemble"
        dir_mod = h_dir / "modelos"
        garantir_pasta(dir_ens)
        garantir_pasta(dir_mod)

        # Ensemble
        out_png_ens = dir_ens / f"MAPA_FDC_{_sanitize_name(fdc_model_ens)}_{h}.png"
        render_map(
            gdf_minis, gdf_sub, gdf_hidro, gdf_cid, gdf_est, code_field, est_fig,
            out_png=out_png_ens, kind="fdc", horizon=h,
            fdc_lookup=fdc_lookup, model=fdc_model_ens,
            ensemble_model_clim=ensemble_model_clim
        )
        nfig += 1
        if nfig % 20 == 0:
            plt.close("all")
            gc.collect()

        # Modelos individuais
        for i_m, m in enumerate(fdc_models_gcm, 1):
            m_safe = _sanitize_name(m)
            out_png_m = dir_mod / f"MAPA_FDC_{m_safe}_{h}.png"
            render_map(
                gdf_minis, gdf_sub, gdf_hidro, gdf_cid, gdf_est, code_field, est_fig,
                out_png=out_png_m, kind="fdc", horizon=h,
                fdc_lookup=fdc_lookup, model=m,
                ensemble_model_clim=ensemble_model_clim
            )
            nfig += 1
            if nfig % 20 == 0:
                plt.close("all")
                gc.collect()
        print(f"      ✓ Ensemble + {len(fdc_models_gcm)} modelos concluídos ({nfig} mapas total)")

    # SAZONALIDADE BOX
    print(f"\n[SAZONAL BOX] Gerando mapas para {len(horizons_box)} horizontes...")
    for i_h, h in enumerate(horizons_box, 1):
        print(f"  [{i_h}/{len(horizons_box)}] Horizonte: {h}")
        h_dir = out_box_root / h
        garantir_pasta(h_dir)
        out_png = h_dir / f"MAPA_SAZ_BOX_{h}.png"
        render_map(
            gdf_minis, gdf_sub, gdf_hidro, gdf_cid, gdf_est, code_field, est_fig,
            out_png=out_png, kind="seasonal_box_gcms", horizon=h,
            season_box_lookup=season_box_lookup,
            ensemble_model_clim=ensemble_model_clim
        )
        nfig += 1
        if nfig % 20 == 0:
            plt.close("all")
            gc.collect()
        print(f"      ✓ Concluído ({nfig} mapas total)")

    # SAZONALIDADE VIOLIN
    print(f"\n[SAZONAL VIOLIN] Gerando mapas para {len(horizons_violin)} horizontes...")
    for i_h, h in enumerate(horizons_violin, 1):
        print(f"  [{i_h}/{len(horizons_violin)}] Horizonte: {h}")
        h_dir = out_violin_root / h
        dir_ens = h_dir / "ensemble"
        dir_mod = h_dir / "modelos"
        garantir_pasta(dir_ens)
        garantir_pasta(dir_mod)

        # Ensemble
        out_png_ens = dir_ens / f"MAPA_SAZ_VIOLIN_ENS_{h}.png"
        render_map(
            gdf_minis, gdf_sub, gdf_hidro, gdf_cid, gdf_est, code_field, est_fig,
            out_png=out_png_ens, kind="seasonal_violin_interannual", horizon=h,
            violin_lookup=violin_lookup, model=violin_model_ens, sim_fonte="ENSEMBLE",
            ensemble_model_clim=ensemble_model_clim
        )
        nfig += 1
        if nfig % 20 == 0:
            plt.close("all")
            gc.collect()

        # Modelos individuais
        for i_m, m in enumerate(violin_models_gcm, 1):
            m_safe = _sanitize_name(m)
            out_png_m = dir_mod / f"MAPA_SAZ_VIOLIN_{m_safe}_{h}.png"
            render_map(
                gdf_minis, gdf_sub, gdf_hidro, gdf_cid, gdf_est, code_field, est_fig,
                out_png=out_png_m, kind="seasonal_violin_interannual", horizon=h,
                violin_lookup=violin_lookup, model=m, sim_fonte="GCM",
                ensemble_model_clim=ensemble_model_clim
            )
            nfig += 1
            if nfig % 20 == 0:
                plt.close("all")
                gc.collect()
        print(f"      ✓ Ensemble + {len(violin_models_gcm)} modelos concluídos ({nfig} mapas total)")

    print("\n" + "="*80)
    print("✅ RODADA COMPLETA FINALIZADA COM SUCESSO!")
    print("="*80)
    print(f"\nEstatísticas finais:")
    print(f"  • Total de mapas gerados: {nfig}")
    print(f"  • Ensemble climatologia: {ensemble_model_clim}")
    print(f"  • Ensemble interanual: {violin_model_ens}")
    print(f"  • Diretório de saída: {OUT_DIR}")
    print()


if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print(f"\n❌ ERRO FATAL: {e}")
        import traceback
        traceback.print_exc()
        sys.exit(1)

In [ ]:
# -*- coding: utf-8 -*-
"""
===============================================================================
COMPARATIVO — FDC + SAZONALIDADE (CENÁRIOS x ABORDAGENS) - LEGENDAS PADRÃO
Legendas no formato: SSP2-4.5 — AB1, SSP2-4.5 — AB2, SSP5-8.5 — AB1, SSP5-8.5 — AB2

CORREÇÃO (REF NÃO ENCONTRADA):
- O filtro de referência estava frágil (dependia de "horizonte" com padrões específicos e, às vezes, de "fonte").
- Em muitos consolidados, a referência pode aparecer como:
  * horizonte diferente do esperado (ex.: "HIST", "HISTORICO", "REF (1980-2023)", etc.)
  * "OBS" em "modelo" (e não em "fonte"), ou até sem coluna "fonte"
  * q_obs_ref preenchido em várias linhas (às vezes repetido por horizonte/modelo)
- Além disso, faltava filtrar também por codigo_mini ao buscar a referência.

Nesta versão:
- Detecta referência por múltiplas regras (horizonte + OBS/fonte/modelo + fallback por q_obs_ref)
- Filtra por (estacao_obs, codigo_mini)
- Mantém o comportamento original de plot, apenas tornando a referência robusta.

===============================================================================
"""

from __future__ import annotations

from pathlib import Path
from typing import Dict, List, Tuple, Optional

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# =============================================================================
# CONFIG
# =============================================================================

OUT_DIR = Path(r"E:\RESULTADOS_COMPARATIVO_FDC_SAZ")
(OUT_DIR / "tables").mkdir(parents=True, exist_ok=True)
(OUT_DIR / "figures" / "FDC").mkdir(parents=True, exist_ok=True)
(OUT_DIR / "figures" / "SAZONAL").mkdir(parents=True, exist_ok=True)

# Mapeamento de cenários para rótulo final
SCN_LABEL = {"SSP2_45": "SSP2-4.5", "SSP5_85": "SSP5-8.5"}

# PALETA DE CORES FIXA - uma cor para cada combinação Cenário-Abordagem
COLOR_PALETTE = {
    "SSP2-4.5 — AB1": "#1f77b4",  # Azul médio
    "SSP2-4.5 — AB2": "#4d9de0",  # Azul claro
    "SSP5-8.5 — AB1": "#d62728",  # Vermelho
    "SSP5-8.5 — AB2": "#ff7f0e",  # Laranja
}

# CONFIGURAÇÃO DA LINHA DE REFERÊNCIA (1980-2023)
REF_COLOR = "#808080"  # Cinza
REF_LINESTYLE = "--"   # Tracejado
REF_LINEWIDTH = 1.5
REF_ALPHA = 0.7
REF_LABEL = "Referência (1980-2023)"
REF_ZORDER = 1  # Atrás das linhas de projeção

# ---- Detecção robusta de referência ----
REF_YEAR_START = 1980
REF_YEAR_END = 2023

# (mantido, mas agora é só uma das pistas)
REF_HORIZONTE_PATTERNS = ["REF_1980_2023", "REF_1980", "1980_2023"]

# Palavras-chave adicionais comuns em consolidados
REF_HZ_KEYWORDS = [
    "REF", "HIST", "HISTOR", "BASE", "OBS_REF", "REFER", "CALIB", "VALID"
]

# OBS em colunas possíveis
OBS_HINTS = ["OBS", "OBSERV", "OBSERVED"]

# Ensemble (nomes podem variar entre AB1/AB2; vamos filtrar por "contém")
FDC_ENSEMBLE_HINTS = ["ENSEMBLE", "MEAN"]      # em "modelo"
SAZ_ENSEMBLE_HINTS = ["ENSEMBLE", "MEAN"]      # em "modelo"
SAZ_FONTE_HINTS    = ["ENSEMBLE"]              # em "fonte"

FDC_PKEYS = [5, 10, 50, 90, 95]  # em %
MONTH_ORDER = list(range(1, 13))
MONTH_LABELS_PT = ["Jan","Fev","Mar","Abr","Mai","Jun","Jul","Ago","Set","Out","Nov","Dez"]

EPS = 1e-9
USE_LOGY_FDC = True

# --- caminhos (use o padrão com "_"; fallback tenta "-") ---
INPUTS = [
    ("AB1", "SSP2_45",
     r"E:\RESULTADOS_AB1\SSP2_45\Sazonalidade\SAZONALIDADE_mensal_consolidada.csv",
     r"E:\RESULTADOS_AB1\SSP2_45\FDC_Projecoes_ssp245\FDC_consolidado_obs_ref_e_futuro.csv"),
    ("AB1", "SSP5_85",
     r"E:\RESULTADOS_AB1\SSP5_85\Sazonalidade\SAZONALIDADE_mensal_consolidada.csv",
     r"E:\RESULTADOS_AB1\SSP5_85\FDC_Projecoes_ssp585\FDC_consolidado_obs_ref_e_futuro.csv"),

    ("AB2", "SSP2_45",
     r"E:\RESULTADOS_AB2\SSP2_45\Sazonalidade\SAZONALIDADE_mensal_consolidada.csv",
     r"E:\RESULTADOS_AB2\SSP2_45\FDC_Projecoes_ssp245\FDC_consolidado_obs_ref_e_futuro.csv"),
    ("AB2", "SSP5_85",
     r"E:\RESULTADOS_AB2\SSP5_85\Sazonalidade\SAZONALIDADE_mensal_consolidada.csv",
     r"E:\RESULTADOS_AB2\SSP5_85\FDC_Projecoes_ssp585\FDC_consolidado_obs_ref_e_futuro.csv"),
]

try:
    from scipy.interpolate import PchipInterpolator
    USE_PCHIP = True
except Exception:
    USE_PCHIP = False


# =============================================================================
# PATH HELPERS
# =============================================================================

def resolve_path(p: str) -> Path:
    """
    Tenta encontrar o arquivo mesmo que você tenha '_' vs '-' no caminho.
    """
    path = Path(p)
    if path.exists():
        return path

    alt = p.replace("SSP2_45", "SSP2-45").replace("SSP5_85", "SSP5-85")
    alt_path = Path(alt)
    if alt_path.exists():
        return alt_path

    alt2 = p.replace("SSP2-45", "SSP2_45").replace("SSP5-85", "SSP5_85")
    alt2_path = Path(alt2)
    if alt2_path.exists():
        return alt2_path

    raise FileNotFoundError(f"Arquivo não encontrado:\n- {p}\n- {alt}\n- {alt2}")


# =============================================================================
# NORMALIZAÇÃO
# =============================================================================

def norm_str(s: pd.Series) -> pd.Series:
    return s.astype(str).str.strip()

def norm_horizon(s: pd.Series) -> pd.Series:
    # remove espaços duplicados, padroniza separadores
    x = s.astype(str).str.strip()
    x = x.str.replace(r"\s+", "_", regex=True)
    x = x.str.replace("-", "_", regex=False)
    return x

def to_int64(series: pd.Series) -> pd.Series:
    return pd.to_numeric(series, errors="coerce").astype("Int64")

def contains_all(text: pd.Series, hints: List[str]) -> pd.Series:
    t = text.astype(str).str.upper()
    m = pd.Series(True, index=text.index)
    for h in hints:
        m &= t.str.contains(h.upper(), na=False)
    return m

def contains_any(text: pd.Series, hints: List[str]) -> pd.Series:
    t = text.astype(str).str.upper()
    m = pd.Series(False, index=text.index)
    for h in hints:
        m |= t.str.contains(h.upper(), na=False)
    return m


# =============================================================================
# LOADERS
# =============================================================================

def load_fdc(path: str, approach: str, scn_code: str) -> pd.DataFrame:
    p = resolve_path(path)
    df = pd.read_csv(p, sep=";")

    out = df.copy()
    out["Approach"] = approach
    out["Scenario"] = SCN_LABEL.get(scn_code, scn_code)

    # normalizações
    for c in ["modelo", "horizonte", "nome_estacao", "fonte"]:
        if c in out.columns:
            out[c] = norm_str(out[c])

    if "horizonte" in out.columns:
        out["horizonte"] = norm_horizon(out["horizonte"])

    out["estacao_obs"] = to_int64(out["estacao_obs"])
    out["codigo_mini"] = to_int64(out["codigo_mini"])

    out["exc"] = pd.to_numeric(out["exc"], errors="coerce")

    # colunas podem ou não existir
    if "q_obs_ref" in out.columns:
        out["q_obs_ref"] = pd.to_numeric(out["q_obs_ref"], errors="coerce")
    else:
        out["q_obs_ref"] = np.nan

    if "q_sim" in out.columns:
        out["q_sim"] = pd.to_numeric(out["q_sim"], errors="coerce")
    else:
        out["q_sim"] = np.nan

    # exige campos mínimos para FDC de projeção (q_sim)
    out = out.dropna(subset=["estacao_obs", "codigo_mini", "horizonte", "exc"])
    # NOTE: não dropa por q_sim aqui, pois OBS/ref pode ter q_obs_ref e q_sim vazio
    return out


def load_saz(path: str, approach: str, scn_code: str) -> pd.DataFrame:
    p = resolve_path(path)
    df = pd.read_csv(p, sep=";")

    out = df.copy()
    out["Approach"] = approach
    out["Scenario"] = SCN_LABEL.get(scn_code, scn_code)

    # normalizações
    for c in ["fonte", "modelo", "horizonte", "nome_estacao"]:
        if c in out.columns:
            out[c] = norm_str(out[c])

    if "horizonte" in out.columns:
        out["horizonte"] = norm_horizon(out["horizonte"])

    out["estacao_obs"] = to_int64(out["estacao_obs"])
    out["codigo_mini"] = to_int64(out["codigo_mini"])

    out["mes"] = to_int64(out["mes"])
    out["Q_medio"] = pd.to_numeric(out["Q_medio"], errors="coerce")

    out = out.dropna(subset=["estacao_obs", "codigo_mini", "horizonte", "mes", "Q_medio"])
    return out


# =============================================================================
# HELPERS — REFERÊNCIA ROBUSTA
# =============================================================================

def is_reference_horizon(hz_series: pd.Series) -> pd.Series:
    """
    Identifica se o horizonte é do período de referência (1980-2023).
    Faz checagens por:
    - padrões antigos (REF_1980_2023 etc.)
    - palavras-chave (REF, HIST...)
    - regex contendo 1980 e 2023 (mesmo com separadores variados)
    """
    hz = hz_series.astype(str).fillna("").str.upper()

    # 1) padrões explícitos (mantido)
    mask = pd.Series(False, index=hz_series.index)
    for pattern in REF_HORIZONTE_PATTERNS:
        mask |= hz.str.contains(pattern.upper(), na=False)

    # 2) palavras-chave comuns
    mask |= contains_any(hz, REF_HZ_KEYWORDS)

    # 3) regex para capturar 1980...2023, aceitando qualquer separador
    # (como já normalizamos '-' para '_' em muitos casos, isso pega ambos)
    mask |= hz.str.contains(rf"{REF_YEAR_START}.*{REF_YEAR_END}", regex=True, na=False)

    return mask


def is_obs_like(df: pd.DataFrame) -> pd.Series:
    """
    Identifica linhas associadas a observado (OBS).
    Tenta em 'fonte' e também em 'modelo' (muitos consolidados usam OBS em modelo).
    Se não existir coluna, retorna False.
    """
    m = pd.Series(False, index=df.index)

    if "fonte" in df.columns:
        m |= contains_any(df["fonte"], OBS_HINTS)

    if "modelo" in df.columns:
        m |= contains_any(df["modelo"], OBS_HINTS)

    return m


def get_reference_fdc_curve(fdc: pd.DataFrame, est: int, mini: int) -> Optional[pd.DataFrame]:
    """
    Retorna curva FDC de referência (exc x Q) para uma estação/minibacia.

    Estratégia (prioridade):
    A) (horizonte referência) AND (OBS-like em fonte/modelo) AND (q_obs_ref disponível)
    B) (horizonte referência) AND (OBS-like) usando q_sim se q_obs_ref não existir
    C) fallback por q_obs_ref preenchido (independente de horizonte/fonte), filtrando est+mini
    """
    df = fdc[(fdc["estacao_obs"] == est) & (fdc["codigo_mini"] == mini)].copy()
    if df.empty:
        return None

    hz_ref = is_reference_horizon(df["horizonte"])
    obs_like = is_obs_like(df)

    # A) referência + OBS + q_obs_ref
    if "q_obs_ref" in df.columns and df["q_obs_ref"].notna().any():
        mA = hz_ref & obs_like & df["q_obs_ref"].notna()
        if mA.any():
            ref = (
                df.loc[mA, ["exc", "q_obs_ref"]]
                .dropna()
                .groupby("exc", as_index=False)["q_obs_ref"].mean()
                .sort_values("exc")
                .rename(columns={"q_obs_ref": "q_ref"})
            )
            if not ref.empty:
                return ref

    # B) referência + OBS usando q_sim
    if "q_sim" in df.columns and df["q_sim"].notna().any():
        mB = hz_ref & obs_like & df["q_sim"].notna()
        if mB.any():
            ref = (
                df.loc[mB, ["exc", "q_sim"]]
                .dropna()
                .groupby("exc", as_index=False)["q_sim"].mean()
                .sort_values("exc")
                .rename(columns={"q_sim": "q_ref"})
            )
            if not ref.empty:
                return ref

    # C) fallback: qualquer linha com q_obs_ref preenchido
    if "q_obs_ref" in df.columns and df["q_obs_ref"].notna().any():
        mC = df["q_obs_ref"].notna()
        ref = (
            df.loc[mC, ["exc", "q_obs_ref"]]
            .dropna()
            .groupby("exc", as_index=False)["q_obs_ref"].mean()
            .sort_values("exc")
            .rename(columns={"q_obs_ref": "q_ref"})
        )
        if not ref.empty:
            return ref

    return None


def get_reference_saz_monthly(saz: pd.DataFrame, est: int, mini: int) -> Optional[pd.DataFrame]:
    """
    Retorna sazonalidade de referência (mes x Q_medio) para uma estação/minibacia.

    Estratégia:
    A) horizonte referência AND OBS-like (fonte/modelo)
    B) fallback: OBS-like independente de horizonte
    """
    df = saz[(saz["estacao_obs"] == est) & (saz["codigo_mini"] == mini)].copy()
    if df.empty:
        return None

    hz_ref = is_reference_horizon(df["horizonte"])
    obs_like = is_obs_like(df)

    mA = hz_ref & obs_like
    if mA.any():
        ref = (
            df.loc[mA, ["mes", "Q_medio"]]
            .dropna()
            .groupby("mes", as_index=False)["Q_medio"].mean()
            .sort_values("mes")
        )
        if not ref.empty:
            return ref

    mB = obs_like
    if mB.any():
        ref = (
            df.loc[mB, ["mes", "Q_medio"]]
            .dropna()
            .groupby("mes", as_index=False)["Q_medio"].mean()
            .sort_values("mes")
        )
        if not ref.empty:
            return ref

    return None


def format_horizonte(hz: str) -> str:
    hz_clean = str(hz).replace("TOTAL_", "").replace("_", "-")
    return f"({hz_clean})"


def pct_change(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    return (b - a) / (np.abs(a) + EPS) * 100.0

def abs_change(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    return (b - a)

def interp_q_at_exc(g: pd.DataFrame, exc_targets: List[float], qcol: str) -> Dict[float, float]:
    g2 = g.sort_values("exc")
    x = g2["exc"].to_numpy()
    y = g2[qcol].to_numpy()
    if len(x) < 2:
        return {pk: np.nan for pk in exc_targets}
    return {pk: float(np.interp(pk, x, y)) for pk in exc_targets}


def build_master_frames() -> Tuple[pd.DataFrame, pd.DataFrame]:
    fdc_all, saz_all = [], []
    for approach, scn_code, saz_path, fdc_path in INPUTS:
        fdc_all.append(load_fdc(fdc_path, approach, scn_code))
        saz_all.append(load_saz(saz_path, approach, scn_code))

    fdc = pd.concat(fdc_all, ignore_index=True)
    saz = pd.concat(saz_all, ignore_index=True)

    # Se SAZ não tem nome_estacao, pegar do FDC
    if "nome_estacao" not in saz.columns or saz["nome_estacao"].isna().all():
        nome_map = fdc[["estacao_obs", "nome_estacao"]].drop_duplicates().dropna()
        nome_map = nome_map.groupby("estacao_obs")["nome_estacao"].first().to_dict()
        saz["nome_estacao"] = saz["estacao_obs"].map(nome_map)
        saz["nome_estacao"] = saz["nome_estacao"].fillna(saz["estacao_obs"].astype(str))

    return fdc, saz


# =============================================================================
# FILTROS ENSEMBLE (ROBUSTOS)
# =============================================================================

def filter_fdc_ensemble(fdc: pd.DataFrame) -> pd.DataFrame:
    # modelo contém ENSEMBLE e MEAN
    if "modelo" not in fdc.columns:
        return fdc.iloc[0:0].copy()
    m = contains_all(fdc["modelo"], FDC_ENSEMBLE_HINTS)
    # para projeção, exigimos q_sim
    out = fdc[m].copy()
    out = out.dropna(subset=["q_sim"])
    return out

def filter_saz_ensemble(saz: pd.DataFrame) -> pd.DataFrame:
    # requer colunas típicas
    if "modelo" not in saz.columns or "fonte" not in saz.columns:
        return saz.iloc[0:0].copy()
    m_model = contains_all(saz["modelo"], SAZ_ENSEMBLE_HINTS)
    m_fonte = contains_all(saz["fonte"], SAZ_FONTE_HINTS)
    return saz[m_model & m_fonte].copy()


# =============================================================================
# TABELAS
# =============================================================================

def summarize_fdc_pkeys(fdc: pd.DataFrame) -> pd.DataFrame:
    f = filter_fdc_ensemble(fdc)

    rows = []
    keys = ["estacao_obs", "codigo_mini", "nome_estacao", "horizonte", "Approach", "Scenario"]
    for grp, g in f.groupby(keys):
        q_map = interp_q_at_exc(g, FDC_PKEYS, "q_sim")
        for pk, qv in q_map.items():
            rows.append((*grp, pk, qv))

    base = pd.DataFrame(rows, columns=keys + ["Pkey", "Q"])

    piv_scn = base.pivot_table(
        index=["estacao_obs","codigo_mini","nome_estacao","horizonte","Approach","Pkey"],
        columns="Scenario", values="Q", aggfunc="mean"
    ).reset_index()

    if ("SSP2-4.5" in piv_scn.columns) and ("SSP5-8.5" in piv_scn.columns):
        a = piv_scn["SSP2-4.5"].to_numpy()
        b = piv_scn["SSP5-8.5"].to_numpy()
        piv_scn["dAbs_SSP585_vs_SSP245"] = abs_change(a, b)
        piv_scn["dPct_SSP585_vs_SSP245"] = pct_change(a, b)
    else:
        piv_scn["dAbs_SSP585_vs_SSP245"] = np.nan
        piv_scn["dPct_SSP585_vs_SSP245"] = np.nan

    out_merge = piv_scn.copy()
    for scn in ["SSP2-4.5", "SSP5-8.5"]:
        tmp = base[base["Scenario"] == scn].pivot_table(
            index=["estacao_obs","codigo_mini","nome_estacao","horizonte","Pkey"],
            columns="Approach", values="Q", aggfunc="mean"
        ).reset_index()

        if ("AB1" in tmp.columns) and ("AB2" in tmp.columns):
            a = tmp["AB1"].to_numpy()
            b = tmp["AB2"].to_numpy()
            tmp[f"dAbs_AB2_vs_AB1_{scn}"] = abs_change(a, b)
            tmp[f"dPct_AB2_vs_AB1_{scn}"] = pct_change(a, b)
        else:
            tmp[f"dAbs_AB2_vs_AB1_{scn}"] = np.nan
            tmp[f"dPct_AB2_vs_AB1_{scn}"] = np.nan

        out_merge = out_merge.merge(
            tmp[["estacao_obs","codigo_mini","nome_estacao","horizonte","Pkey",
                 f"dAbs_AB2_vs_AB1_{scn}", f"dPct_AB2_vs_AB1_{scn}"]],
            on=["estacao_obs","codigo_mini","nome_estacao","horizonte","Pkey"], how="left"
        )

    return out_merge.sort_values(["estacao_obs","horizonte","Approach","Pkey"])


def summarize_saz_monthly(saz: pd.DataFrame) -> pd.DataFrame:
    s = filter_saz_ensemble(saz)

    base = s[["estacao_obs","codigo_mini","horizonte","Approach","Scenario","mes","Q_medio"]].copy()

    piv_scn = base.pivot_table(
        index=["estacao_obs","codigo_mini","horizonte","Approach","mes"],
        columns="Scenario", values="Q_medio", aggfunc="mean"
    ).reset_index()

    if ("SSP2-4.5" in piv_scn.columns) and ("SSP5-8.5" in piv_scn.columns):
        a = piv_scn["SSP2-4.5"].to_numpy()
        b = piv_scn["SSP5-8.5"].to_numpy()
        piv_scn["dAbs_SSP585_vs_SSP245"] = abs_change(a, b)
        piv_scn["dPct_SSP585_vs_SSP245"] = pct_change(a, b)
    else:
        piv_scn["dAbs_SSP585_vs_SSP245"] = np.nan
        piv_scn["dPct_SSP585_vs_SSP245"] = np.nan

    out_merge = piv_scn.copy()
    for scn in ["SSP2-4.5", "SSP5-8.5"]:
        tmp = base[base["Scenario"] == scn].pivot_table(
            index=["estacao_obs","codigo_mini","horizonte","mes"],
            columns="Approach", values="Q_medio", aggfunc="mean"
        ).reset_index()

        if ("AB1" in tmp.columns) and ("AB2" in tmp.columns):
            a = tmp["AB1"].to_numpy()
            b = tmp["AB2"].to_numpy()
            tmp[f"dAbs_AB2_vs_AB1_{scn}"] = abs_change(a, b)
            tmp[f"dPct_AB2_vs_AB1_{scn}"] = pct_change(a, b)
        else:
            tmp[f"dAbs_AB2_vs_AB1_{scn}"] = np.nan
            tmp[f"dPct_AB2_vs_AB1_{scn}"] = np.nan

        out_merge = out_merge.merge(
            tmp[["estacao_obs","codigo_mini","horizonte","mes",
                 f"dAbs_AB2_vs_AB1_{scn}", f"dPct_AB2_vs_AB1_{scn}"]],
            on=["estacao_obs","codigo_mini","horizonte","mes"], how="left"
        )

    return out_merge.sort_values(["estacao_obs","horizonte","Approach","mes"])


# =============================================================================
# FIGURAS - COM LEGENDAS PADRONIZADAS
# =============================================================================

def _apply_logy_if_possible(ax, series_list: List[pd.Series]) -> None:
    if not USE_LOGY_FDC:
        return
    ok = True
    for s in series_list:
        if s is None or len(s) == 0:
            continue
        if np.nanmin(s.to_numpy()) <= 0:
            ok = False
            break
    if ok:
        ax.set_yscale("log")


def plot_fdc_compare(fdc: pd.DataFrame) -> None:
    f = filter_fdc_ensemble(fdc)

    out_dir = OUT_DIR / "figures" / "FDC"
    out_dir.mkdir(parents=True, exist_ok=True)

    id_cols = ["estacao_obs","codigo_mini","nome_estacao"]
    for (est, mini, nome), g0 in f.groupby(id_cols):

        # ---- REFERÊNCIA ROBUSTA (agora filtra est+mini e usa fallbacks) ----
        ref_curve = get_reference_fdc_curve(fdc, int(est), int(mini))

        if ref_curve is None or ref_curve.empty:
            # Debug útil (sem confundir com máscaras globais)
            hz_list = (
                fdc[(fdc["estacao_obs"] == est) & (fdc["codigo_mini"] == mini)]["horizonte"]
                .dropna().unique()
            )
            print(f"\n[FDC] Estação {est} ({nome}) mini={mini}")
            print("  ✗ Referência NÃO encontrada (após fallbacks).")
            print("  - Horizontes (est+mini):")
            for hz in hz_list:
                print(f"    - {hz}")
            if "fonte" in fdc.columns:
                fontes = fdc[(fdc["estacao_obs"] == est) & (fdc["codigo_mini"] == mini)]["fonte"].dropna().unique()
                print(f"  - Fontes (est+mini): {list(fontes)[:20]}")
            if "modelo" in fdc.columns:
                modelos = fdc[(fdc["estacao_obs"] == est) & (fdc["codigo_mini"] == mini)]["modelo"].dropna().unique()
                # só imprime alguns
                print(f"  - Modelos (est+mini) [amostra]: {list(modelos)[:10]}")
        else:
            print(f"\n[FDC] Estação {est} ({nome}) mini={mini} — ✓ Referência OK: {len(ref_curve)} pontos")

        for hz, gh in g0.groupby("horizonte"):

            fig = plt.figure(figsize=(13, 10))
            axs = [fig.add_subplot(2,2,i+1) for i in range(4)]

            # (1) AB1: cenários
            ax = axs[0]
            series_for_log = []

            if ref_curve is not None and not ref_curve.empty:
                ax.plot(
                    ref_curve["exc"], ref_curve["q_ref"],
                    color=REF_COLOR, linestyle=REF_LINESTYLE, linewidth=REF_LINEWIDTH,
                    alpha=REF_ALPHA, label=REF_LABEL, zorder=REF_ZORDER
                )
                series_for_log.append(ref_curve["q_ref"])

            for scn in ["SSP2-4.5","SSP5-8.5"]:
                gg = gh[(gh["Approach"]=="AB1") & (gh["Scenario"]==scn)].sort_values("exc")
                if gg.empty:
                    continue
                label = f"{scn} — AB1"
                ax.plot(gg["exc"], gg["q_sim"], linewidth=2, label=label, color=COLOR_PALETTE[label], zorder=3)
                series_for_log.append(gg["q_sim"])

            _apply_logy_if_possible(ax, series_for_log)
            ax.set_title("AB1 — SSP2-4.5 vs SSP5-8.5", fontweight='bold')
            ax.set_xlabel("Excedência (%)")
            ax.set_ylabel("Q (m³/s)")
            ax.grid(True, alpha=0.3, which="both")
            ax.legend()

            # (2) AB2: cenários
            ax = axs[1]
            series_for_log = []

            if ref_curve is not None and not ref_curve.empty:
                ax.plot(
                    ref_curve["exc"], ref_curve["q_ref"],
                    color=REF_COLOR, linestyle=REF_LINESTYLE, linewidth=REF_LINEWIDTH,
                    alpha=REF_ALPHA, label=REF_LABEL, zorder=REF_ZORDER
                )
                series_for_log.append(ref_curve["q_ref"])

            for scn in ["SSP2-4.5","SSP5-8.5"]:
                gg = gh[(gh["Approach"]=="AB2") & (gh["Scenario"]==scn)].sort_values("exc")
                if gg.empty:
                    continue
                label = f"{scn} — AB2"
                ax.plot(gg["exc"], gg["q_sim"], linewidth=2, label=label, color=COLOR_PALETTE[label], zorder=3)
                series_for_log.append(gg["q_sim"])

            _apply_logy_if_possible(ax, series_for_log)
            ax.set_title("AB2 — SSP2-4.5 vs SSP5-8.5", fontweight='bold')
            ax.set_xlabel("Excedência (%)")
            ax.set_ylabel("Q (m³/s)")
            ax.grid(True, alpha=0.3, which="both")
            ax.legend()

            # (3) SSP2-4.5: abordagens
            ax = axs[2]
            series_for_log = []

            if ref_curve is not None and not ref_curve.empty:
                ax.plot(
                    ref_curve["exc"], ref_curve["q_ref"],
                    color=REF_COLOR, linestyle=REF_LINESTYLE, linewidth=REF_LINEWIDTH,
                    alpha=REF_ALPHA, label=REF_LABEL, zorder=REF_ZORDER
                )
                series_for_log.append(ref_curve["q_ref"])

            for ab in ["AB1","AB2"]:
                gg = gh[(gh["Approach"]==ab) & (gh["Scenario"]=="SSP2-4.5")].sort_values("exc")
                if gg.empty:
                    continue
                label = f"SSP2-4.5 — {ab}"
                ax.plot(gg["exc"], gg["q_sim"], linewidth=2, label=label, color=COLOR_PALETTE[label], zorder=3)
                series_for_log.append(gg["q_sim"])

            _apply_logy_if_possible(ax, series_for_log)
            ax.set_title("SSP2-4.5 — AB1 vs AB2", fontweight='bold')
            ax.set_xlabel("Excedência (%)")
            ax.set_ylabel("Q (m³/s)")
            ax.grid(True, alpha=0.3, which="both")
            ax.legend()

            # (4) SSP5-8.5: abordagens
            ax = axs[3]
            series_for_log = []

            if ref_curve is not None and not ref_curve.empty:
                ax.plot(
                    ref_curve["exc"], ref_curve["q_ref"],
                    color=REF_COLOR, linestyle=REF_LINESTYLE, linewidth=REF_LINEWIDTH,
                    alpha=REF_ALPHA, label=REF_LABEL, zorder=REF_ZORDER
                )
                series_for_log.append(ref_curve["q_ref"])

            for ab in ["AB1","AB2"]:
                gg = gh[(gh["Approach"]==ab) & (gh["Scenario"]=="SSP5-8.5")].sort_values("exc")
                if gg.empty:
                    continue
                label = f"SSP5-8.5 — {ab}"
                ax.plot(gg["exc"], gg["q_sim"], linewidth=2, label=label, color=COLOR_PALETTE[label], zorder=3)
                series_for_log.append(gg["q_sim"])

            _apply_logy_if_possible(ax, series_for_log)
            ax.set_title("SSP5-8.5 — AB1 vs AB2", fontweight='bold')
            ax.set_xlabel("Excedência (%)")
            ax.set_ylabel("Q (m³/s)")
            ax.grid(True, alpha=0.3, which="both")
            ax.legend()

            fig.suptitle(
                f"FDC (Ensemble) — Estação {nome} | {est} | Horizonte {format_horizonte(hz)}",
                y=0.98, fontsize=14, fontweight='bold'
            )
            fig.tight_layout(rect=[0, 0, 1, 0.965])

            fn = out_dir / f"FDC_COMP_{est}_{mini}_{hz}.png"
            fig.savefig(fn, dpi=300)
            plt.close(fig)


def plot_saz_compare(saz: pd.DataFrame) -> None:
    s = filter_saz_ensemble(saz)

    out_dir = OUT_DIR / "figures" / "SAZONAL"
    out_dir.mkdir(parents=True, exist_ok=True)

    for (est, mini, nome_est), g0 in s.groupby(["estacao_obs","codigo_mini","nome_estacao"]):

        # ---- REFERÊNCIA ROBUSTA (agora filtra est+mini e usa fallbacks) ----
        ref_monthly = get_reference_saz_monthly(saz, int(est), int(mini))

        if ref_monthly is None or ref_monthly.empty:
            hz_list = (
                saz[(saz["estacao_obs"] == est) & (saz["codigo_mini"] == mini)]["horizonte"]
                .dropna().unique()
            )
            print(f"\n[SAZ] Estação {est} ({nome_est}) mini={mini}")
            print("  ✗ Referência NÃO encontrada (após fallbacks).")
            print("  - Horizontes (est+mini):")
            for hz in hz_list:
                print(f"    - {hz}")
            if "fonte" in saz.columns:
                fontes = saz[(saz["estacao_obs"] == est) & (saz["codigo_mini"] == mini)]["fonte"].dropna().unique()
                print(f"  - Fontes (est+mini): {list(fontes)[:20]}")
            if "modelo" in saz.columns:
                modelos = saz[(saz["estacao_obs"] == est) & (saz["codigo_mini"] == mini)]["modelo"].dropna().unique()
                print(f"  - Modelos (est+mini) [amostra]: {list(modelos)[:10]}")
        else:
            print(f"\n[SAZ] Estação {est} ({nome_est}) mini={mini} — ✓ Referência OK: {len(ref_monthly)} meses")

        for hz, gh in g0.groupby("horizonte"):

            fig = plt.figure(figsize=(13, 10))
            axs = [fig.add_subplot(2,2,i+1) for i in range(4)]

            # função auxiliar para plotar a referência (suavizada)
            def _plot_ref(ax_):
                if ref_monthly is None or ref_monthly.empty:
                    return
                x_ref = ref_monthly["mes"].to_numpy()
                y_ref = ref_monthly["Q_medio"].to_numpy()

                if USE_PCHIP and len(x_ref) >= 4:
                    x_dense = np.linspace(1, 12, 200)
                    y_dense = PchipInterpolator(x_ref, y_ref)(x_dense)
                    ax_.plot(
                        x_dense, y_dense,
                        color=REF_COLOR, linestyle=REF_LINESTYLE,
                        linewidth=REF_LINEWIDTH, alpha=REF_ALPHA,
                        label=REF_LABEL, zorder=REF_ZORDER
                    )
                    ax_.scatter(
                        x_ref, y_ref,
                        color=REF_COLOR, edgecolor="black",
                        linewidth=0.6, s=25, alpha=REF_ALPHA, zorder=REF_ZORDER
                    )
                else:
                    ax_.plot(
                        x_ref, y_ref,
                        marker="o", color=REF_COLOR, linestyle=REF_LINESTYLE,
                        linewidth=REF_LINEWIDTH, alpha=REF_ALPHA,
                        label=REF_LABEL, zorder=REF_ZORDER
                    )

            # (1) AB1: cenários
            ax = axs[0]
            _plot_ref(ax)

            for scn in ["SSP2-4.5","SSP5-8.5"]:
                gg = gh[(gh["Approach"]=="AB1") & (gh["Scenario"]==scn)].sort_values("mes")
                if gg.empty:
                    continue
                x = gg["mes"].to_numpy()
                y = gg["Q_medio"].to_numpy()
                label = f"{scn} — AB1"
                color = COLOR_PALETTE[label]

                if USE_PCHIP and len(x) >= 4:
                    x_dense = np.linspace(1, 12, 200)
                    y_dense = PchipInterpolator(x, y)(x_dense)
                    ax.plot(x_dense, y_dense, linewidth=2, label=label, color=color, zorder=3)
                    ax.scatter(x, y, color=color, edgecolor="black", linewidth=0.6, s=35, zorder=3)
                else:
                    ax.plot(x, y, marker="o", linewidth=2, label=label, color=color, zorder=3)

            ax.set_title("AB1 — SSP2-4.5 vs SSP5-8.5", fontweight='bold')
            ax.set_xlabel("Mês")
            ax.set_ylabel("Q média mensal (m³/s)")
            ax.set_xticks(MONTH_ORDER)
            ax.set_xticklabels(MONTH_LABELS_PT)
            ax.grid(True, alpha=0.3)
            ax.legend()

            # (2) AB2: cenários
            ax = axs[1]
            _plot_ref(ax)

            for scn in ["SSP2-4.5","SSP5-8.5"]:
                gg = gh[(gh["Approach"]=="AB2") & (gh["Scenario"]==scn)].sort_values("mes")
                if gg.empty:
                    continue
                x = gg["mes"].to_numpy()
                y = gg["Q_medio"].to_numpy()
                label = f"{scn} — AB2"
                color = COLOR_PALETTE[label]

                if USE_PCHIP and len(x) >= 4:
                    x_dense = np.linspace(1, 12, 200)
                    y_dense = PchipInterpolator(x, y)(x_dense)
                    ax.plot(x_dense, y_dense, linewidth=2, label=label, color=color, zorder=3)
                    ax.scatter(x, y, color=color, edgecolor="black", linewidth=0.6, s=35, zorder=3)
                else:
                    ax.plot(x, y, marker="o", linewidth=2, label=label, color=color, zorder=3)

            ax.set_title("AB2 — SSP2-4.5 vs SSP5-8.5", fontweight='bold')
            ax.set_xlabel("Mês")
            ax.set_ylabel("Q média mensal (m³/s)")
            ax.set_xticks(MONTH_ORDER)
            ax.set_xticklabels(MONTH_LABELS_PT)
            ax.grid(True, alpha=0.3)
            ax.legend()

            # (3) SSP2-4.5: abordagens
            ax = axs[2]
            _plot_ref(ax)

            for ab in ["AB1","AB2"]:
                gg = gh[(gh["Approach"]==ab) & (gh["Scenario"]=="SSP2-4.5")].sort_values("mes")
                if gg.empty:
                    continue
                x = gg["mes"].to_numpy()
                y = gg["Q_medio"].to_numpy()
                label = f"SSP2-4.5 — {ab}"
                color = COLOR_PALETTE[label]

                if USE_PCHIP and len(x) >= 4:
                    x_dense = np.linspace(1, 12, 200)
                    y_dense = PchipInterpolator(x, y)(x_dense)
                    ax.plot(x_dense, y_dense, linewidth=2, label=label, color=color, zorder=3)
                    ax.scatter(x, y, color=color, edgecolor="black", linewidth=0.6, s=35, zorder=3)
                else:
                    ax.plot(x, y, marker="o", linewidth=2, label=label, color=color, zorder=3)

            ax.set_title("SSP2-4.5 — AB1 vs AB2", fontweight='bold')
            ax.set_xlabel("Mês")
            ax.set_ylabel("Q média mensal (m³/s)")
            ax.set_xticks(MONTH_ORDER)
            ax.set_xticklabels(MONTH_LABELS_PT)
            ax.grid(True, alpha=0.3)
            ax.legend()

            # (4) SSP5-8.5: abordagens
            ax = axs[3]
            _plot_ref(ax)

            for ab in ["AB1","AB2"]:
                gg = gh[(gh["Approach"]==ab) & (gh["Scenario"]=="SSP5-8.5")].sort_values("mes")
                if gg.empty:
                    continue
                x = gg["mes"].to_numpy()
                y = gg["Q_medio"].to_numpy()
                label = f"SSP5-8.5 — {ab}"
                color = COLOR_PALETTE[label]

                if USE_PCHIP and len(x) >= 4:
                    x_dense = np.linspace(1, 12, 200)
                    y_dense = PchipInterpolator(x, y)(x_dense)
                    ax.plot(x_dense, y_dense, linewidth=2, label=label, color=color, zorder=3)
                    ax.scatter(x, y, color=color, edgecolor="black", linewidth=0.6, s=35, zorder=3)
                else:
                    ax.plot(x, y, marker="o", linewidth=2, label=label, color=color, zorder=3)

            ax.set_title("SSP5-8.5 — AB1 vs AB2", fontweight='bold')
            ax.set_xlabel("Mês")
            ax.set_ylabel("Q média mensal (m³/s)")
            ax.set_xticks(MONTH_ORDER)
            ax.set_xticklabels(MONTH_LABELS_PT)
            ax.grid(True, alpha=0.3)
            ax.legend()

            fig.suptitle(
                f"Vazão média mensal (Ensemble) — Estação {nome_est} | {est} | Horizonte {format_horizonte(hz)}",
                y=0.98, fontsize=14, fontweight='bold'
            )
            fig.tight_layout(rect=[0, 0, 1, 0.965])

            fn = out_dir / f"SAZ_COMP_{est}_{mini}_{hz}.png"
            fig.savefig(fn, dpi=300)
            plt.close(fig)


# =============================================================================
# MAIN
# =============================================================================

def main() -> None:
    print("="*80)
    print("COMPARATIVO FDC + SAZONALIDADE - LEGENDAS PADRONIZADAS")
    print("Formato: SSP2-4.5 — AB1, SSP2-4.5 — AB2, SSP5-8.5 — AB1, SSP5-8.5 — AB2")
    print("="*80)

    fdc, saz = build_master_frames()

    print(f"\n[INFO] FDC carregado: {len(fdc)} linhas")
    print(f"[INFO] SAZ carregado: {len(saz)} linhas")

    # Verificar dados antes de processar
    print("\n[INFO] Combinações FDC disponíveis:")
    if "Approach" in fdc.columns and "Scenario" in fdc.columns:
        print(fdc.groupby(["Approach","Scenario"]).size())
    else:
        print("  (colunas Approach/Scenario ausentes)")

    print("\n[INFO] Combinações SAZ disponíveis:")
    if "Approach" in saz.columns and "Scenario" in saz.columns:
        print(saz.groupby(["Approach","Scenario"]).size())
    else:
        print("  (colunas Approach/Scenario ausentes)")

    # tabelas
    print("\n[INFO] Gerando tabelas resumo...")
    fdc_sum = summarize_fdc_pkeys(fdc)
    saz_sum = summarize_saz_monthly(saz)

    fdc_sum.to_csv(OUT_DIR / "tables" / "fdc_pkeys_summary.csv", index=False, encoding="utf-8-sig")
    saz_sum.to_csv(OUT_DIR / "tables" / "saz_monthly_summary.csv", index=False, encoding="utf-8-sig")

    fdc_sum.to_excel(OUT_DIR / "tables" / "fdc_pkeys_summary.xlsx", index=False)
    saz_sum.to_excel(OUT_DIR / "tables" / "saz_monthly_summary.xlsx", index=False)
    print(f"  -> Tabelas salvas em {OUT_DIR / 'tables'}")

    # figuras
    print("\n[INFO] Gerando figuras FDC...")
    plot_fdc_compare(fdc)

    print("\n[INFO] Gerando figuras Sazonalidade...")
    plot_saz_compare(saz)

    print("\n" + "="*80)
    print(f"✓ CONCLUÍDO — Todas as saídas em: {OUT_DIR}")
    print("="*80)


if __name__ == "__main__":
    main()